## 난임 환자 대상 임신 성공 여부 예측

### LGAimers 6th 온라인 해커톤

Import

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import os
from pprint import pprint

In [3]:
import platform
from matplotlib import rc

# 운영 체제에 따라 폰트 설정
if platform.system() == 'Windows':  # Windows 환경
    rc('font', family='Malgun Gothic')  # 맑은 고딕

# 음수 표시 가능하도록 설정
plt.rcParams['axes.unicode_minus'] = False

반복적으로 쓰는 함수툴

In [4]:
def plot_box(df, col_name):
    """
    주어진 데이터프레임과 열 이름에 대해 박스 플롯을 그리는 함수.

    Parameters:
    df (pd.DataFrame): 데이터프레임
    col_name (str): 열 이름
    """
    plt.figure(figsize=(7, 4))
    plt.boxplot(df[col_name], vert=False)
    plt.xlabel(col_name)
    plt.title(f'Box Plot of {col_name}')
    plt.show()

def value_counts_ratio(df, col_name, target_name='임신_성공_여부'):
    """
    주어진 데이터프레임의 특정 열에 대해 각 값마다 타겟 변수의 비율과 갯수, 총 갯수를 출력하는 함수.

    Parameters:
    df (pd.DataFrame): 데이터프레임
    col_name (str): 열 이름
    target_name (str): 타겟 변수 이름
    """
    # 각 값마다 타겟 변수의 비율 계산
    value_counts = df.groupby(col_name)[target_name].value_counts(normalize=True).unstack().fillna(0)
    
    # 각 값마다 타겟 변수의 갯수 계산
    counts = df.groupby(col_name)[target_name].value_counts().unstack().fillna(0)
    
    # 각 값마다 총 갯수 계산
    total_counts = df[col_name].value_counts().rename('Total_Count')
    
    # 비율과 갯수를 합침
    result = value_counts.join(counts, lsuffix='_ratio', rsuffix='_count')
    
    # 총 갯수를 합침
    result = result.join(total_counts, on=col_name)
    
    # 출력 형식 조정
    result.index.name = 'variable'
    print(f"\n{col_name}별 {target_name} 비율 및 갯수\n")
    print(result.rename(columns=lambda x: x.split('_')[0]))

def summarize_group(df, group_by_columns):
    """
    주어진 데이터프레임을 특정 열들로 그룹화하여 각 그룹의 '임신_성공_여부' 비율과 갯수를 요약하는 함수.

    Parameters:
    df (pd.DataFrame): 데이터프레임
    group_by_columns (list): 그룹화할 열 이름들의 리스트
    """
    # 데이터프레임을 그룹화
    grouped_df = df.groupby(group_by_columns)
    
    # 결과를 저장할 리스트 초기화
    results = []
    
    # 그룹화된 데이터프레임의 내용을 확인하는 코드
    for name, group in grouped_df:
        # 그룹의 갯수 계산
        group_count = group.shape[0]
        
        # '임신_성공_여부' 변수의 '임신 성공' 비율과 갯수 계산
        success_count = group['임신_성공_여부'].value_counts().get(1, 0)
        success_ratio = success_count / group_count
        
        # 결과 리스트에 추가
        results.append([name, success_count, success_ratio, group_count])
    
    # 결과 리스트를 데이터프레임으로 변환
    results_df = pd.DataFrame(results, columns=['group', "'임신_성공_여부' count", 'ratio', 'Total'])
    
    # 그룹화된 변수들의 이름을 제목행으로 출력
    print(f"Grouped by: {', '.join(group_by_columns)}")
    print()
    # 데이터프레임 출력
    print(results_df)

def plot_ratio(df, group_by_column, target_column='임신_성공_여부', success_value=1):
    """
    주어진 데이터프레임을 특정 열로 그룹화하여 각 그룹의 타겟 변수의 성공 비율을 시각화하는 함수.

    Parameters:
    df (pd.DataFrame): 데이터프레임
    group_by_column (str): 그룹화할 열 이름
    target_column (str): 타겟 변수 이름
    success_value (int): 타겟 변수의 성공을 나타내는 값
    """
    # 데이터프레임을 그룹화
    grouped_df = df.groupby(group_by_column)
    
    # 결과를 저장할 리스트 초기화
    results = []
    
    # 그룹화된 데이터프레임의 내용을 확인하는 코드
    for name, group in grouped_df:
        # 그룹의 갯수 계산
        group_count = group.shape[0]
        
        # '임신_성공_여부' 변수의 '성공' 비율과 갯수 계산
        success_count = group[target_column].value_counts().get(success_value, 0)
        success_ratio = success_count / group_count
        
        # 결과 리스트에 추가
        results.append([name, success_count, success_ratio, group_count])
    
    # 결과 리스트를 데이터프레임으로 변환
    results_df = pd.DataFrame(results, columns=['group', f"'{success_value}' count", 'ratio', 'Total'])
    
    # 그래프 크기 설정
    plt.figure(figsize=(10, 5))
    
    # 막대 그래프 생성
    ax = results_df.plot(kind='bar', x='group', y='ratio', legend=False)
    
    # 각 막대 위에 임신 성공 여부 갯수와 총 갯수 표시
    for i, (success_count, total) in enumerate(zip(results_df[f"'{success_value}' count"], results_df['Total'])):
        ax.text(i, results_df['ratio'][i], f'{success_count} ({total})', ha='center', va='bottom', fontsize=8)
    
    # 그래프 제목 및 축 레이블 설정
    ax.set_title(f'{success_value} Ratio by {group_by_column}')
    ax.set_xlabel(group_by_column)
    ax.set_ylabel(f'{success_value} Ratio')
   
    # 그래프 출력
    plt.show()

def plot_ratio_boxplot(data, column_name, target_column='임신_성공_여부'):
    """
    주어진 데이터프레임의 특정 열과 타겟 변수 간의 관계를 박스 플롯으로 시각화하는 함수.

    Parameters:
    data (pd.DataFrame): 데이터프레임
    column_name (str): 열 이름
    target_column (str): 타겟 변수 이름
    """

    # 그래프 그리기
    plt.figure(figsize=(10, 6))
    sns.boxplot(x=column_name, y=target_column, data=data)

    # 그래프 제목 및 레이블 설정
    plt.title(f'{column_name} vs {target_column}')
    plt.xlabel(column_name)
    plt.ylabel(target_column)

    # 그래프 출력
    plt.show()

### Data Load

In [5]:
# 데이터 로드
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')

# 변수명에서 공백을 밑줄로 대체
train.columns = train.columns.str.replace(' ', '_')
test.columns = test.columns.str.replace(' ', '_')

# 변경된 변수명 확인
print(train.columns)

Index(['ID', '시술_시기_코드', '시술_당시_나이', '임신_시도_또는_마지막_임신_경과_연수', '시술_유형',
       '특정_시술_유형', '배란_자극_여부', '배란_유도_유형', '단일_배아_이식_여부', '착상_전_유전_검사_사용_여부',
       '착상_전_유전_진단_사용_여부', '남성_주_불임_원인', '남성_부_불임_원인', '여성_주_불임_원인',
       '여성_부_불임_원인', '부부_주_불임_원인', '부부_부_불임_원인', '불명확_불임_원인', '불임_원인_-_난관_질환',
       '불임_원인_-_남성_요인', '불임_원인_-_배란_장애', '불임_원인_-_여성_요인', '불임_원인_-_자궁경부_문제',
       '불임_원인_-_자궁내막증', '불임_원인_-_정자_농도', '불임_원인_-_정자_면역학적_요인',
       '불임_원인_-_정자_운동성', '불임_원인_-_정자_형태', '배아_생성_주요_이유', '총_시술_횟수',
       '클리닉_내_총_시술_횟수', 'IVF_시술_횟수', 'DI_시술_횟수', '총_임신_횟수', 'IVF_임신_횟수',
       'DI_임신_횟수', '총_출산_횟수', 'IVF_출산_횟수', 'DI_출산_횟수', '총_생성_배아_수',
       '미세주입된_난자_수', '미세주입에서_생성된_배아_수', '이식된_배아_수', '미세주입_배아_이식_수', '저장된_배아_수',
       '미세주입_후_저장된_배아_수', '해동된_배아_수', '해동_난자_수', '수집된_신선_난자_수', '저장된_신선_난자_수',
       '혼합된_난자_수', '파트너_정자와_혼합된_난자_수', '기증자_정자와_혼합된_난자_수', '난자_출처', '정자_출처',
       '난자_기증자_나이', '정자_기증자_나이', '동결_배아_사용_여부', '신선_배아_사용_여부', '기증_배아_사용_여부',
       '대리모_여부', 'PGD_시술_여부', 'PGS_시술_여부

In [6]:
train.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256351 entries, 0 to 256350
Data columns (total 69 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   ID                     256351 non-null  object 
 1   시술_시기_코드               256351 non-null  object 
 2   시술_당시_나이               256351 non-null  object 
 3   임신_시도_또는_마지막_임신_경과_연수  9370 non-null    float64
 4   시술_유형                  256351 non-null  object 
 5   특정_시술_유형               256349 non-null  object 
 6   배란_자극_여부               256351 non-null  int64  
 7   배란_유도_유형               256351 non-null  object 
 8   단일_배아_이식_여부            250060 non-null  float64
 9   착상_전_유전_검사_사용_여부       2718 non-null    float64
 10  착상_전_유전_진단_사용_여부       250060 non-null  float64
 11  남성_주_불임_원인             256351 non-null  int64  
 12  남성_부_불임_원인             256351 non-null  int64  
 13  여성_주_불임_원인             256351 non-null  int64  
 14  여성_부_불임_원인             256351 non-nu

In [7]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90067 entries, 0 to 90066
Data columns (total 68 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   ID                     90067 non-null  object 
 1   시술_시기_코드               90067 non-null  object 
 2   시술_당시_나이               90067 non-null  object 
 3   임신_시도_또는_마지막_임신_경과_연수  3297 non-null   float64
 4   시술_유형                  90067 non-null  object 
 5   특정_시술_유형               90067 non-null  object 
 6   배란_자극_여부               90067 non-null  int64  
 7   배란_유도_유형               90067 non-null  object 
 8   단일_배아_이식_여부            87891 non-null  float64
 9   착상_전_유전_검사_사용_여부       933 non-null    float64
 10  착상_전_유전_진단_사용_여부       87891 non-null  float64
 11  남성_주_불임_원인             90067 non-null  int64  
 12  남성_부_불임_원인             90067 non-null  int64  
 13  여성_주_불임_원인             90067 non-null  int64  
 14  여성_부_불임_원인             90067 non-null  int64  
 15  부부

In [8]:
# '임신_성공_여부' 변수의 값 개수 계산
value_counts = train['임신_성공_여부'].value_counts()

# 각 값이 전체에서 차지하는 비율 계산
value_percentages = value_counts / len(train) * 100

# 결과 출력
print("임신_성공_여부 값 개수:")
print(value_counts)
print("\n임신_성공_여부 값 비율 (%):")
print(value_percentages)

임신_성공_여부 값 개수:
임신_성공_여부
0    190123
1     66228
Name: count, dtype: int64

임신_성공_여부 값 비율 (%):
임신_성공_여부
0    74.16511
1    25.83489
Name: count, dtype: float64


무조건 불임인 행 따로 구분해둠

In [9]:
# '시술_당시_나이' 변수의 값이 '알 수 없음'인 행 제거
train_sub = train[train['시술_당시_나이'] == '알 수 없음']
train = train[train['시술_당시_나이'] != '알 수 없음']

# '남성_주_불임_원인'과 '남성_부_불임_원인'의 값이 동시에 1인 행 제거
train_sub = pd.concat([train_sub, train[(train['남성_주_불임_원인'] == 1) & (train['남성_부_불임_원인'] == 1)]])
train = train[~((train['남성_주_불임_원인'] == 1) & (train['남성_부_불임_원인'] == 1))]

# '여성_주_불임_원인'과 '여성_부_불임_원인'의 값이 동시에 1인 행 제거
train_sub = pd.concat([train_sub, train[(train['여성_주_불임_원인'] == 1) & (train['여성_부_불임_원인'] == 1)]])
train = train[~((train['여성_주_불임_원인'] == 1) & (train['여성_부_불임_원인'] == 1))]

# '부부_주_불임_원인'과 '부부_부_불임_원인'의 값이 동시에 1인 행 제거
train_sub = pd.concat([train_sub, train[(train['부부_주_불임_원인'] == 1) & (train['부부_부_불임_원인'] == 1)]])
train = train[~((train['부부_주_불임_원인'] == 1) & (train['부부_부_불임_원인'] == 1))]

## EDA

### 1. 시술_시기_코드

난임 시술을 받은 시기를 기준으로 코드 부여

In [10]:
value_counts_ratio(train, '시술_시기_코드')


시술_시기_코드별 임신_성공_여부 비율 및 갯수

                 0         1      0     1  Total
variable                                        
TRCMWS    0.742198  0.257802  28228  9805  38033
TRDQAZ    0.754095  0.245905  29329  9564  38893
TRJXFG    0.733306  0.266694  26377  9593  35970
TRVNRY    0.739976  0.260024  26742  9397  36139
TRXQMD    0.743715  0.256285  25885  8920  34805
TRYBLT    0.730480  0.269520  26775  9879  36654
TRZKPL    0.744664  0.255336  26446  9068  35514


In [11]:
# '시술_시기_코드' 변수 제거
train = train.drop(columns=['시술_시기_코드'])
test = test.drop(columns=['시술_시기_코드'])

### 2. 시술 당시 나이

환자의 시술 당시 나이(연령대)

In [12]:
value_counts_ratio(train, '시술_당시_나이')


시술_당시_나이별 임신_성공_여부 비율 및 갯수

                 0         1      0      1   Total
variable                                          
만18-34세   0.677378  0.322622  69415  33061  102476
만35-37세   0.721585  0.278415  41691  16086   57777
만38-39세   0.782849  0.217151  30719   8521   39240
만40-42세   0.840599  0.159401  31393   5953   37346
만43-44세   0.882050  0.117950  10806   1445   12251
만45-50세   0.832321  0.167679   5758   1160    6918


In [13]:
# '시술_당시_나이' 변수의 값을 중앙값으로 변환하는 함수 정의
def convert_age_category(age_category):
    if age_category == '만18-34세':
        return (18 + 34) / 2
    elif age_category == '만35-37세':
        return (35 + 37) / 2
    elif age_category == '만38-39세':
        return (38 + 39) / 2
    elif age_category == '만40-42세':
        return (40 + 42) / 2
    elif age_category == '만43-44세':
        return (43 + 44) / 2
    elif age_category == '만45-50세':
        return (45 + 50) / 2
    # elif age_category == '알 수 없음':
        # return -1
    else:
        return None

train['시술_당시_나이'] = train['시술_당시_나이'].apply(convert_age_category)
test['시술_당시_나이'] = test['시술_당시_나이'].apply(convert_age_category)

In [14]:
train['시술_당시_나이'].value_counts()

시술_당시_나이
26.0    102476
36.0     57777
38.5     39240
41.0     37346
43.5     12251
47.5      6918
Name: count, dtype: int64

### 3. 임신 시도 또는 마지막 임신 경과 일수

환자가 처음 임신을 시도한 시점 또는 마지막 임신 이후 현재까지의 경과 연수 (년 단위)

In [15]:
value_counts_ratio(train, '임신_시도_또는_마지막_임신_경과_연수')


임신_시도_또는_마지막_임신_경과_연수별 임신_성공_여부 비율 및 갯수

                 0         1    0    1  Total
variable                                     
0.0       0.666667  0.333333    8    4     12
1.0       0.666667  0.333333    4    2      6
2.0       0.809524  0.190476   17    4     21
3.0       0.752809  0.247191   67   22     89
4.0       0.756579  0.243421  230   74    304
5.0       0.753623  0.246377  520  170    690
6.0       0.750253  0.249747  742  247    989
7.0       0.763636  0.236364  924  286   1210
8.0       0.784864  0.215136  923  253   1176
9.0       0.796477  0.203523  814  208   1022
10.0      0.778247  0.221753  737  210    947
11.0      0.810271  0.189729  568  133    701
12.0      0.780392  0.219608  398  112    510
13.0      0.837838  0.162162  403   78    481
14.0      0.776435  0.223565  257   74    331
15.0      0.791339  0.208661  201   53    254
16.0      0.846535  0.153465  171   31    202
17.0      0.820144  0.179856  114   25    139
18.0      0.839623  0.160377   89   17

In [16]:
print("결측값 개수:", train['임신_시도_또는_마지막_임신_경과_연수'].isnull().sum())

train['임신_시도_또는_마지막_임신_경과_연수'].fillna(-1, inplace=True)
test['임신_시도_또는_마지막_임신_경과_연수'].fillna(-1, inplace=True)

print("결측값 개수:", train['임신_시도_또는_마지막_임신_경과_연수'].isnull().sum())

결측값 개수: 246649
결측값 개수: 0


In [17]:
value_counts_ratio(train, '임신_시도_또는_마지막_임신_경과_연수')


임신_시도_또는_마지막_임신_경과_연수별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
-1.0      0.739727  0.260273  182453  64196  246649
 0.0      0.666667  0.333333       8      4      12
 1.0      0.666667  0.333333       4      2       6
 2.0      0.809524  0.190476      17      4      21
 3.0      0.752809  0.247191      67     22      89
 4.0      0.756579  0.243421     230     74     304
 5.0      0.753623  0.246377     520    170     690
 6.0      0.750253  0.249747     742    247     989
 7.0      0.763636  0.236364     924    286    1210
 8.0      0.784864  0.215136     923    253    1176
 9.0      0.796477  0.203523     814    208    1022
 10.0     0.778247  0.221753     737    210     947
 11.0     0.810271  0.189729     568    133     701
 12.0     0.780392  0.219608     398    112     510
 13.0     0.837838  0.162162     403     78     481
 14.0     0.776435  0.223565     257     74     331
 15.0     0.791339  0.

### 4. 시술_유형

IVF 또는 DI 시술 여부

In [18]:
value_counts_ratio(train, '시술_유형')


시술_유형별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
DI        0.871065  0.128935    5479    811    6290
IVF       0.738045  0.261955  184303  65415  249718


In [19]:
test['시술_유형'].value_counts()

시술_유형
IVF    87891
DI      2176
Name: count, dtype: int64

In [20]:
# 시술_유형에 따라 파생변수 생성
train['시술_유형_IVF'] = np.where(train['시술_유형'] == 'IVF', 1, 0)
train['시술_유형_DI'] = np.where(train['시술_유형'] == 'DI', 1, 0)

test['시술_유형_IVF'] = np.where(test['시술_유형'] == 'IVF', 1, 0)
test['시술_유형_DI'] = np.where(test['시술_유형'] == 'DI', 1, 0)

In [21]:
summarize_group(train, ['시술_유형_DI','시술_유형_IVF'])

Grouped by: 시술_유형_DI, 시술_유형_IVF

    group  '임신_성공_여부' count     ratio   Total
0  (0, 1)             65415  0.261955  249718
1  (1, 0)               811  0.128935    6290


In [22]:
train = train.drop(columns=['시술_유형'])
test = test.drop(columns=['시술_유형'])

### 5. 특정_시술_유형

IVF – 체외 수정 (시험관 아기 시술)  
ICSI – 세포질 내 정자 주입  
IUI – 인공 수정 (자궁 내 주입)  
ICI – 자궁경부 인공 수정  
GIFT – 생식세포 나팔관 이식  
FER – 냉동 배아 이식  
Generic DI – 기증 정자 주입  
IVI – 질 내 인공 수정   
BLASTOCYST – 배반포 단계에 이식  
AH – 배아를 hatching 시켜주는 과정  
Unknown – 시술 유형 정보가 제공되지 않았거나 알 수 없음  

(*) 특수 기호의 의미  
"/" – 복합적인 시술 방법을 의미 (예: ICSI / BLASTOCYST)  
":" – 동일 시술 내 세부 조합을 의미 (예: ICSI:IVF)  

In [23]:
value_counts_ratio(train, '특정_시술_유형')


특정_시술_유형별 임신_성공_여부 비율 및 갯수

                                            0         1        0        1  \
variable                                                                    
FER                                  0.666667  0.333333      2.0      1.0   
GIFT                                 1.000000  0.000000      2.0      0.0   
Generic DI                           0.830986  0.169014     59.0     12.0   
ICI                                  0.864583  0.135417     83.0     13.0   
ICSI                                 0.727137  0.272863  88966.0  33385.0   
ICSI / AH                            0.782835  0.217165    602.0    167.0   
ICSI / AH:Unknown                    0.500000  0.500000      1.0      1.0   
ICSI / BLASTOCYST                    0.643257  0.356743   1035.0    574.0   
ICSI / BLASTOCYST :ICSI              0.000000  1.000000      0.0      1.0   
ICSI / BLASTOCYST :IVF / BLASTOCYST  0.750000  0.250000      3.0      1.0   
ICSI / BLASTOCYST:IVF / BLASTOCYST   0.666667  

In [24]:
test['특정_시술_유형'].value_counts()

특정_시술_유형
ICSI                                  43237
IVF                                   32350
Unknown                                9216
IUI                                    2112
ICSI:ICSI                               797
ICSI / BLASTOCYST                       584
IVF:IVF                                 406
IVF / BLASTOCYST                        389
ICSI:IVF                                298
ICSI / AH                               264
IVF:ICSI                                133
IVF / AH                                105
ICSI:Unknown                             68
IVF:Unknown                              41
ICI                                      28
Generic DI                               26
IVI                                       8
ICSI / BLASTOCYST:IVF / BLASTOCYST        2
IUI:ICI                                   2
IVF:Unknown:Unknown:Unknown               1
Name: count, dtype: int64

In [25]:
# '특정_시술_유형' 변수의 값이 'FER' 또는 'GIFT'인 행 제거
train = train.loc[~train['특정_시술_유형'].isin(['FER', 'GIFT'])]
# test = test.loc[~test['특정_시술_유형'].isin(['FER', 'GIFT'])]

In [26]:
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer

# 시술 유형 수정 함수 정의
def clean_procedure(procedure):
    if isinstance(procedure, str):
        # 공백 제거
        procedure = procedure.replace(' ', '')
        # 시술 뒤에 Unknown이 붙어있으면 제거
        if ':Unknown' in procedure:
            procedure = procedure.replace(':Unknown', '')
    return procedure

# 시술 유형 수정
train['특정_시술_유형'] = train['특정_시술_유형'].apply(clean_procedure)
test['특정_시술_유형'] = test['특정_시술_유형'].apply(clean_procedure)

# 시술 유형 분리 및 원핫 인코딩 함수 정의
def process_procedures(df):
    # 시술 유형 분리
    df_split = df['특정_시술_유형'].apply(lambda x: x.split(':') if isinstance(x, str) else [])
    
    # '/'로 구분된 시술 유형을 개별 항목으로 분리
    df_split = df_split.apply(lambda x: [item for sublist in [i.split('/') for i in x] for item in sublist])
    
    # MultiLabelBinarizer를 사용하여 원핫 인코딩
    mlb = MultiLabelBinarizer()
    reasons_encoded = mlb.fit_transform(df_split)
    reasons_df = pd.DataFrame(reasons_encoded, columns=[f'특정_시술_유형_{cls}' for cls in mlb.classes_])
    
    # '특정_시술_유형_Unknown' 변수 제거
    if '특정_시술_유형_Unknown' in reasons_df.columns:
        reasons_df = reasons_df.drop(columns=['특정_시술_유형_Unknown'])
    
    # 원래 데이터프레임과 병합
    df_encoded = df.drop(columns=['특정_시술_유형']).reset_index(drop=True)
    df_encoded = pd.concat([df_encoded, reasons_df], axis=1)
    
    return df_encoded

train = process_procedures(train)
test = process_procedures(test)

In [27]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256003 entries, 0 to 256002
Data columns (total 76 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   ID                     256003 non-null  object 
 1   시술_당시_나이               256003 non-null  float64
 2   임신_시도_또는_마지막_임신_경과_연수  256003 non-null  float64
 3   배란_자극_여부               256003 non-null  int64  
 4   배란_유도_유형               256003 non-null  object 
 5   단일_배아_이식_여부            249714 non-null  float64
 6   착상_전_유전_검사_사용_여부       2718 non-null    float64
 7   착상_전_유전_진단_사용_여부       249714 non-null  float64
 8   남성_주_불임_원인             256003 non-null  int64  
 9   남성_부_불임_원인             256003 non-null  int64  
 10  여성_주_불임_원인             256003 non-null  int64  
 11  여성_부_불임_원인             256003 non-null  int64  
 12  부부_주_불임_원인             256003 non-null  int64  
 13  부부_부_불임_원인             256003 non-null  int64  
 14  불명확_불임_원인              256003 non-nu

### 6. 배란_자극_여부

배란 자극 치료가 사용되었는지 여부

In [28]:
value_counts_ratio(train, '배란_자극_여부')


배란_자극_여부별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
0         0.768344  0.231656   45038  13579   58617
1         0.733284  0.266716  144740  52646  197386


### 7. 배란_유도_유형

사용된 배란 유도 방법 유형

In [29]:
value_counts_ratio(train, '배란_유도_유형')


배란_유도_유형별 임신_성공_여부 비율 및 갯수

                     0         1         0        1   Total
variable                                                   
기록되지 않은 시행    0.731133  0.268867  141913.0  52187.0  194100
생식선 자극 호르몬    0.000000  1.000000       0.0      1.0       1
세트로타이드 (억제제)  1.000000  0.000000       1.0      0.0       1
알 수 없음        0.773235  0.226765   47864.0  14037.0   61901


In [30]:
test['배란_유도_유형'].value_counts()

배란_유도_유형
기록되지 않은 시행    68599
알 수 없음        21468
Name: count, dtype: int64

In [31]:
# 해당하는 행이 적은 '생식선 자극 호르몬', '세트로타이드 (억제제)' 제거
train = train.loc[~train['배란_유도_유형'].isin(['생식선 자극 호르몬', '세트로타이드 (억제제)'])]
test = test.loc[~test['배란_유도_유형'].isin(['생식선 자극 호르몬', '세트로타이드 (억제제)'])]

In [32]:
value_counts_ratio(train, '배란_유도_유형')


배란_유도_유형별 임신_성공_여부 비율 및 갯수

                   0         1       0      1   Total
variable                                             
기록되지 않은 시행  0.731133  0.268867  141913  52187  194100
알 수 없음      0.773235  0.226765   47864  14037   61901


In [33]:
# '배란_유도_유형' 변수 제거
train = train.drop(columns=['배란_유도_유형'])
test = test.drop(columns=['배란_유도_유형'])

### 8. 단일_배아_이식_여부

선택적으로 단일 배아를 이식했는지 여부

In [34]:
value_counts_ratio(train, '단일_배아_이식_여부')


단일_배아_이식_여부별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
0.0       0.770172  0.229828  147357  43973  191330
1.0       0.632764  0.367236   36942  21440   58382


In [35]:
# 결측값 개수 확인
print("결측값 개수:", train['단일_배아_이식_여부'].isnull().sum())

# 결측값을 -1 으로 채우기
train['단일_배아_이식_여부'].fillna(-1, inplace=True)
test['단일_배아_이식_여부'].fillna(-1, inplace=True)

# 결측값 개수 확인
print("결측값 개수:", train['단일_배아_이식_여부'].isnull().sum())

결측값 개수: 6289
결측값 개수: 0


### 9. 착상_전_유전_검사_사용_여부

PGS(착상 전 유전 검사)가 시행되었는지 여부

In [36]:
value_counts_ratio(train, '착상_전_유전_검사_사용_여부')


착상_전_유전_검사_사용_여부별 임신_성공_여부 비율 및 갯수

                 0         1     0    1  Total
variable                                      
1.0       0.851729  0.148271  2315  403   2718


In [37]:
# 결측값 개수 확인
print("결측값 개수:", train['착상_전_유전_검사_사용_여부'].isnull().sum())

# 결측값을 0으로 채우기
train['착상_전_유전_검사_사용_여부'].fillna(0, inplace=True)
test['착상_전_유전_검사_사용_여부'].fillna(0, inplace=True)

# 결측값 개수 확인
print("결측값 개수:", train['착상_전_유전_검사_사용_여부'].isnull().sum())

결측값 개수: 253283
결측값 개수: 0


In [38]:
value_counts_ratio(train, '착상_전_유전_검사_사용_여부')


착상_전_유전_검사_사용_여부별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
0.0       0.740129  0.259871  187462  65821  253283
1.0       0.851729  0.148271    2315    403    2718


### 10. 착상_전_유전_진단_사용_여부

PGD(착상 전 유전 진단)가 시행되었는지 여부

In [39]:
value_counts_ratio(train, '착상_전_유전_진단_사용_여부')


착상_전_유전_진단_사용_여부별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
0.0       0.737141  0.262859  181717  64799  246516
1.0       0.807885  0.192115    2582    614    3196


In [40]:
# 결측값 개수 확인
print("결측값 개수:", train['착상_전_유전_진단_사용_여부'].isnull().sum())

# 결측값을 0으로 채우기
train['착상_전_유전_진단_사용_여부'].fillna(-1, inplace=True)
test['착상_전_유전_진단_사용_여부'].fillna(-1, inplace=True)

# 결측값 개수 확인
print("결측값 개수:", train['착상_전_유전_진단_사용_여부'].isnull().sum())

결측값 개수: 6289
결측값 개수: 0


In [41]:
value_counts_ratio(train, '착상_전_유전_진단_사용_여부')


착상_전_유전_진단_사용_여부별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
-1.0      0.871045  0.128955    5478    811    6289
 0.0      0.737141  0.262859  181717  64799  246516
 1.0      0.807885  0.192115    2582    614    3196


In [42]:
summarize_group(train, ['착상_전_유전_진단_사용_여부', '착상_전_유전_검사_사용_여부'])

Grouped by: 착상_전_유전_진단_사용_여부, 착상_전_유전_검사_사용_여부

         group  '임신_성공_여부' count     ratio   Total
0  (-1.0, 0.0)               811  0.128955    6289
1   (0.0, 0.0)             64398  0.264117  243824
2   (0.0, 1.0)               401  0.148960    2692
3   (1.0, 0.0)               612  0.193060    3170
4   (1.0, 1.0)                 2  0.076923      26


### 11. 남성_주_불임_원인

남성의 주요 불임 원인 여부

In [43]:
value_counts_ratio(train, '남성_주_불임_원인')


남성_주_불임_원인별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
0         0.740177  0.259823  184082  64618  248700
1         0.780030  0.219970    5695   1606    7301


### 12. 남성_부_불임_원인

남성의 부차적 불임 원인 여부

In [44]:
value_counts_ratio(train, '남성_부_불임_원인')


남성_부_불임_원인별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
0         0.740504  0.259496  187086  65561  252647
1         0.802326  0.197674    2691    663    3354


In [45]:
summarize_group(train, ['남성_주_불임_원인', '남성_부_불임_원인'])

Grouped by: 남성_주_불임_원인, 남성_부_불임_원인

    group  '임신_성공_여부' count     ratio   Total
0  (0, 0)             63955  0.260673  245346
1  (0, 1)               663  0.197674    3354
2  (1, 0)              1606  0.219970    7301


### 13. 여성_주_불임_원인

여성의 주요 불임 원인 여부

In [46]:
value_counts_ratio(train, '여성_주_불임_원인')


여성_주_불임_원인별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
0         0.740057  0.259943  183634  64501  248135
1         0.780956  0.219044    6143   1723    7866


### 14. 여성_부_불임_원인

여성의 부차적 불임 원인 여부

In [47]:
value_counts_ratio(train, '여성_부_불임_원인')


여성_부_불임_원인별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
0         0.740526  0.259474  187225  65602  252827
1         0.804033  0.195967    2552    622    3174


In [48]:
summarize_group(train, ['여성_주_불임_원인', '여성_부_불임_원인'])

Grouped by: 여성_주_불임_원인, 여성_부_불임_원인

    group  '임신_성공_여부' count     ratio   Total
0  (0, 0)             63879  0.260772  244961
1  (0, 1)               622  0.195967    3174
2  (1, 0)              1723  0.219044    7866


### 15. 부부_주_불임_원인

부부의 주요 불임 원인 여부

In [49]:
value_counts_ratio(train, '부부_주_불임_원인')


부부_주_불임_원인별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
0         0.740001  0.259999  183179  64360  247539
1         0.779721  0.220279    6598   1864    8462


### 16. 부부_부_불임_원인

부부의 부차적 불임 원인 여부

In [50]:
value_counts_ratio(train, '부부_부_불임_원인')


부부_부_불임_원인별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
0         0.740684  0.259316  187956  65804  253760
1         0.812584  0.187416    1821    420    2241


In [51]:
summarize_group(train, ['부부_주_불임_원인', '부부_부_불임_원인'])

Grouped by: 부부_주_불임_원인, 부부_부_불임_원인

    group  '임신_성공_여부' count     ratio   Total
0  (0, 0)             63940  0.260663  245298
1  (0, 1)               420  0.187416    2241
2  (1, 0)              1864  0.220279    8462


### 17. 불명확_불임_원인

불명의 원인으로 인한 불임 여부

In [52]:
value_counts_ratio(train, '불명확_불임_원인')


불명확_불임_원인별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
0         0.741578  0.258422  142182  49547  191729
1         0.740525  0.259475   47595  16677   64272


### 18. 불임_원인_-_난관_질환

난관 이상으로 인한 불임 여부

In [53]:
value_counts_ratio(train, '불임_원인_-_난관_질환')


불임_원인_-_난관_질환별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
0         0.741165  0.258835  163389  57060  220449
1         0.742237  0.257763   26388   9164   35552


### 19. 불임_원인_-_남성_요인

남성 요인으로 인한 불임 여부

In [54]:
value_counts_ratio(train, '불임_원인_-_남성_요인')


불임_원인_-_남성_요인별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
0         0.753923  0.246077  121040  39507  160547
1         0.720106  0.279894   68737  26717   95454


### 20. 불임_원인_-_배란_장애

배란 장애로 인한 불임 여부

In [55]:
value_counts_ratio(train, '불임_원인_-_배란_장애')


불임_원인_-_배란_장애별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
0         0.745511  0.254489  165932  56643  222575
1         0.713367  0.286633   23845   9581   33426


### 21. 불임_원인_-_여성_요인

여성 요인으로 인한 불임 여부

In [56]:
value_counts_ratio(train, '불임_원인_-_여성_요인')


불임_원인_-_여성_요인별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
0         0.741314  0.258686  189777  66224  256001


In [57]:
test['불임_원인_-_여성_요인'].value_counts()

불임_원인_-_여성_요인
0    90067
Name: count, dtype: int64

In [58]:
train['불임_원인_-_여성_요인'].value_counts()

불임_원인_-_여성_요인
0    256001
Name: count, dtype: int64

In [59]:
train['불임_원인_-_여성_요인'].isnull().sum()

0

In [60]:
# '불임_원인_-_여성_요인' 변수 제거
train = train.drop(columns=['불임_원인_-_여성_요인'])
test = test.drop(columns=['불임_원인_-_여성_요인'])

### 22. 불임_원인_-_자궁경부_문제

자궁경부 이상으로 인한 불임 여부

In [61]:
value_counts_ratio(train, '불임_원인_-_자궁경부_문제')


불임_원인_-_자궁경부_문제별 임신_성공_여부 비율 및 갯수

                 0         1         0        1   Total
variable                                               
0         0.741303  0.258697  189767.0  66224.0  255991
1         1.000000  0.000000      10.0      0.0      10


### 23. 불임_원인_-_자궁내막증

자궁내막증으로 인한 불임 여부

In [62]:
value_counts_ratio(train, '불임_원인_-_자궁내막증')


불임_원인_-_자궁내막증별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
0         0.741017  0.258983  176139  61560  237699
1         0.745164  0.254836   13638   4664   18302


### 24. 불임_원인_-_정자_농도

파트너의 정자 농도 문제 여부

In [63]:
value_counts_ratio(train, '불임_원인_-_정자_농도')


불임_원인_-_정자_농도별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
0         0.741304  0.258696  189570  66155  255725
1         0.750000  0.250000     207     69     276


### 25. 불임_원인_-_정자_면역학적_요인

파트너의 정자 면역 관련 문제 여부

In [64]:
value_counts_ratio(train, '불임_원인_-_정자_면역학적_요인')


불임_원인_-_정자_면역학적_요인별 임신_성공_여부 비율 및 갯수

                 0         1         0        1   Total
variable                                               
0         0.741313  0.258688  189776.0  66224.0  256000
1         1.000000  0.000000       1.0      0.0       1


In [65]:
test['불임_원인_-_정자_면역학적_요인'].value_counts()

불임_원인_-_정자_면역학적_요인
0    90066
1        1
Name: count, dtype: int64

### 26. 불임_원인_-_정자_운동성

파트너의 정자 운동성 문제 여부

In [66]:
value_counts_ratio(train, '불임_원인_-_정자_운동성')


불임_원인_-_정자_운동성별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
0         0.741282  0.258718  189697  66207  255904
1         0.824742  0.175258      80     17      97


### 27. 불임_원인_-_정자_형태

파트너의 정자 형태 문제 여부

In [67]:
value_counts_ratio(train, '불임_원인_-_정자_형태')


불임_원인_-_정자_형태별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
0         0.741294  0.258706  189666  66192  255858
1         0.776224  0.223776     111     32     143


In [68]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 256001 entries, 0 to 256002
Data columns (total 74 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   ID                     256001 non-null  object 
 1   시술_당시_나이               256001 non-null  float64
 2   임신_시도_또는_마지막_임신_경과_연수  256001 non-null  float64
 3   배란_자극_여부               256001 non-null  int64  
 4   단일_배아_이식_여부            256001 non-null  float64
 5   착상_전_유전_검사_사용_여부       256001 non-null  float64
 6   착상_전_유전_진단_사용_여부       256001 non-null  float64
 7   남성_주_불임_원인             256001 non-null  int64  
 8   남성_부_불임_원인             256001 non-null  int64  
 9   여성_주_불임_원인             256001 non-null  int64  
 10  여성_부_불임_원인             256001 non-null  int64  
 11  부부_주_불임_원인             256001 non-null  int64  
 12  부부_부_불임_원인             256001 non-null  int64  
 13  불명확_불임_원인              256001 non-null  int64  
 14  불임_원인_-_난관_질환          256001 non-null  i

In [ ]:
summarize_group(train, ['불임_원인_-_난관_질환'
])

### 28. 배아_생성_주요_이유

배아 생성의 주된 목적(시술, 보관 등)

In [1642]:
value_counts_ratio(train, '배아_생성_주요_이유')


배아_생성_주요_이유별 임신_성공_여부 비율 및 갯수

                            0         1         0        1   Total
variable                                                          
기증용                  1.000000  0.000000    1108.0      0.0    1108
기증용, 난자 저장용          1.000000  0.000000      44.0      0.0      44
기증용, 배아 저장용          1.000000  0.000000     125.0      0.0     125
기증용, 배아 저장용, 현재 시술용  0.650000  0.350000      13.0      7.0      20
기증용, 현재 시술용          0.620243  0.379757    2347.0   1437.0    3784
난자 저장용               1.000000  0.000000    1959.0      0.0    1959
난자 저장용, 배아 저장용       1.000000  0.000000       6.0      0.0       6
난자 저장용, 배아 저장용, 연구용  1.000000  0.000000       1.0      0.0       1
난자 저장용, 현재 시술용       0.800000  0.200000       4.0      1.0       5
배아 저장용               0.999130  0.000870    9183.0      8.0    9191
배아 저장용, 현재 시술용       0.734940  0.265060      61.0     22.0      83
연구용, 현재 시술용          1.000000  0.000000       1.0      0.0       1
현재 시술용               0.726433 

In [1643]:
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer

# 배아_생성_주요_이유 분리 함수 정의
def split_reasons(reason):
    if isinstance(reason, str):
        return reason.split(', ')
    return []

# MultiLabelBinarizer를 사용하여 원핫 인코딩
def process_reasons(df):
    mlb = MultiLabelBinarizer()
    reasons = df['배아_생성_주요_이유'].apply(split_reasons)
    reasons_encoded = mlb.fit_transform(reasons)
    reasons_df = pd.DataFrame(reasons_encoded, columns=[f'배아_생성_주요_이유_{cls}' for cls in mlb.classes_])
    df = df.drop(columns=['배아_생성_주요_이유']).reset_index(drop=True)
    df_encoded = pd.concat([df, reasons_df], axis=1)
    
    # '배아_생성_주요_이유_연구용' 변수 제거
    if '배아_생성_주요_이유_연구용' in df_encoded.columns:
        df_encoded = df_encoded.drop(columns=['배아_생성_주요_이유_연구용'])
    
    return df_encoded

train = process_reasons(train)
test = process_reasons(test)

In [1644]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256344 entries, 0 to 256343
Data columns (total 77 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   ID                     256344 non-null  object 
 1   시술_당시_나이               256344 non-null  object 
 2   임신_시도_또는_마지막_임신_경과_연수  256344 non-null  float64
 3   배란_자극_여부               256344 non-null  int64  
 4   단일_배아_이식_여부            256344 non-null  float64
 5   착상_전_유전_검사_사용_여부       256344 non-null  float64
 6   착상_전_유전_진단_사용_여부       256344 non-null  float64
 7   남성_주_불임_원인             256344 non-null  int64  
 8   남성_부_불임_원인             256344 non-null  int64  
 9   여성_주_불임_원인             256344 non-null  int64  
 10  여성_부_불임_원인             256344 non-null  int64  
 11  부부_주_불임_원인             256344 non-null  int64  
 12  부부_부_불임_원인             256344 non-null  int64  
 13  불명확_불임_원인              256344 non-null  int64  
 14  불임_원인_-_난관_질환          256344 non-nu

### 29. 총_시술_횟수

IVF 및 DI 포함한 전체 시술 횟수

In [1645]:
value_counts_ratio(train, '총_시술_횟수')


총_시술_횟수별 임신_성공_여부 비율 및 갯수

                 0         1      0      1  Total
variable                                         
0회        0.709010  0.290990  69198  28400  97598
1회        0.750053  0.249947  42615  14201  56816
2회        0.755853  0.244147  29733   9604  39337
3회        0.763646  0.236354  18733   5798  24531
4회        0.772208  0.227792  11692   3449  15141
5회        0.784208  0.215792   7141   1965   9106
6회 이상     0.796670  0.203330  11006   2809  13815


### 30. 클리닉_내_총_시술_횟수

특정 클리닉에서 받은 IVF 및 DI 시술 횟수

In [1646]:
value_counts_ratio(train, '클리닉_내_총_시술_횟수')


클리닉_내_총_시술_횟수별 임신_성공_여부 비율 및 갯수

                 0         1      0      1   Total
variable                                          
0회        0.717080  0.282920  87250  34424  121674
1회        0.754624  0.245376  45088  14661   59749
2회        0.762073  0.237927  26338   8223   34561
3회        0.767555  0.232445  14090   4267   18357
4회        0.778099  0.221901   7795   2223   10018
5회        0.792439  0.207561   4276   1120    5396
6회 이상     0.801487  0.198513   5281   1308    6589


### 31. IVF_시술_횟수

이전에 수행된 IVF 시술 횟수

In [1647]:
value_counts_ratio(train, 'IVF_시술_횟수')


IVF_시술_횟수별 임신_성공_여부 비율 및 갯수

                 0         1      0      1   Total
variable                                          
0회        0.713739  0.286261  74181  29752  103933
1회        0.749434  0.250566  43719  14617   58336
2회        0.754563  0.245437  29560   9615   39175
3회        0.764476  0.235524  17797   5483   23280
4회        0.772548  0.227452  10553   3107   13660
5회        0.787234  0.212766   6142   1660    7802
6회 이상     0.803898  0.196102   8166   1992   10158


### 32. DI_시술_횟수

이전에 수행된 DI 시술 횟수

In [1648]:
value_counts_ratio(train, 'DI_시술_횟수')


DI_시술_횟수별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
0회        0.739349  0.260651  179261  63197  242458
1회        0.807877  0.192123    2359    561    2920
2회        0.782768  0.217232    2353    653    3006
3회        0.757702  0.242298    2361    755    3116
4회        0.779386  0.220614    1346    381    1727
5회        0.811823  0.188177     975    226    1201
6회 이상     0.763570  0.236430    1463    453    1916


### 33. 총_임신_횟수

IVF 및 DI 포함한 총 임신 횟수

In [1649]:
value_counts_ratio(train, '총_임신_횟수')


총_임신_횟수별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
0회        0.742547  0.257453  152534  52886  205420
1회        0.737092  0.262908   32306  11523   43829
2회        0.742075  0.257925    4635   1611    6246
3회        0.754692  0.245308     563    183     746
4회        0.782609  0.217391      72     20      92
5회        0.750000  0.250000       6      2       8
6회 이상     0.666667  0.333333       2      1       3


### 34. IVF_임신_횟수

IVF 시술로 인한 임신 횟수

In [1650]:
value_counts_ratio(train, 'IVF_임신_횟수')


IVF_임신_횟수별 임신_성공_여부 비율 및 갯수

                 0         1         0        1   Total
variable                                               
0회        0.743151  0.256849  154788.0  53498.0  208286
1회        0.733592  0.266408   30458.0  11061.0   41519
2회        0.743953  0.256047    4306.0   1482.0    5788
3회        0.752640  0.247360     499.0    164.0     663
4회        0.762500  0.237500      61.0     19.0      80
5회        0.857143  0.142857       6.0      1.0       7
6회 이상     0.000000  1.000000       0.0      1.0       1


### 35. DI_임신_횟수

DI 시술로 인한 임신 횟수

In [1651]:
value_counts_ratio(train, 'DI_임신_횟수')


DI_임신_횟수별 임신_성공_여부 비율 및 갯수

                 0         1         0        1   Total
variable                                               
0회        0.741152  0.258848  187730.0  65565.0  253295
1회        0.790857  0.209143    2076.0    549.0    2625
2회        0.725543  0.274457     267.0    101.0     368
3회        0.791667  0.208333      38.0     10.0      48
4회        1.000000  0.000000       5.0      0.0       5
5회        0.666667  0.333333       2.0      1.0       3


### 36. 총_출산_횟수

IVF 및 DI 시술로 출산한 횟수

In [1652]:
value_counts_ratio(train, '총_출산_횟수')


총_출산_횟수별 임신_성공_여부 비율 및 갯수

                 0         1         0        1   Total
variable                                               
0회        0.743151  0.256849  162415.0  56134.0  218549
1회        0.732753  0.267247   25916.0   9452.0   35368
2회        0.736842  0.263158    1652.0    590.0    2242
3회        0.721893  0.278107     122.0     47.0     169
4회        0.769231  0.230769      10.0      3.0      13
5회        1.000000  0.000000       1.0      0.0       1
6회 이상     1.000000  0.000000       2.0      0.0       2


### 37. IVF_출산_횟수

IVF 시술로 출산한 횟수

In [1653]:
value_counts_ratio(train, 'IVF_출산_횟수')


IVF_출산_횟수별 임신_성공_여부 비율 및 갯수

                 0         1         0        1   Total
variable                                               
0회        0.743457  0.256543  164174.0  56651.0  220825
1회        0.730111  0.269889   24347.0   9000.0   33347
2회        0.737259  0.262741    1490.0    531.0    2021
3회        0.700730  0.299270      96.0     41.0     137
4회        0.769231  0.230769      10.0      3.0      13
5회        1.000000  0.000000       1.0      0.0       1


### 38. DI_출산_횟수

DI 시술로 출산한 횟수

In [1654]:
value_counts_ratio(train, 'DI_출산_횟수')


DI_출산_횟수별 임신_성공_여부 비율 및 갯수

                 0         1         0        1   Total
variable                                               
0회        0.741384  0.258616  188313.0  65689.0  254002
1회        0.770814  0.229186    1648.0    490.0    2138
2회        0.765027  0.234973     140.0     43.0     183
3회        0.789474  0.210526      15.0      4.0      19
5회        1.000000  0.000000       2.0      0.0       2


In [1655]:
# 숫자 매핑
mapping = {
    '0회': 0, '1회': 1, '2회': 2, '3회': 3, 
    '4회': 4, '5회': 5, '6회 이상': 6
}

# 변환할 열 목록
columns_to_map = [
    '총_시술_횟수', '클리닉_내_총_시술_횟수', 'IVF_시술_횟수', 
    'DI_시술_횟수', '총_임신_횟수', 'IVF_임신_횟수', 'DI_임신_횟수', 
    '총_출산_횟수', 'IVF_출산_횟수', 'DI_출산_횟수'
]

# 숫자로 변환
for col in columns_to_map:
    train[col] = train[col].map(mapping)
    test[col] = test[col].map(mapping)

In [1656]:
# 비율 계산 (train 데이터프레임)
train['IVF_임신_시술_비율'] = np.where(train['IVF_시술_횟수'] > 0, train['IVF_임신_횟수'] / train['IVF_시술_횟수'], -1)
train['DI_임신_시술_비율'] = np.where(train['DI_시술_횟수'] > 0, train['DI_임신_횟수'] / train['DI_시술_횟수'], -1)
train['임신_성공률'] = np.where(train['총_시술_횟수'] > 0, train['총_임신_횟수'] / train['총_시술_횟수'], -1)

# 비율 계산 (test 데이터프레임)
test['IVF_임신_시술_비율'] = np.where(test['IVF_시술_횟수'] > 0, test['IVF_임신_횟수'] / test['IVF_시술_횟수'], -1)
test['DI_임신_시술_비율'] = np.where(test['DI_시술_횟수'] > 0, test['DI_임신_횟수'] / test['DI_시술_횟수'], -1)
test['임신_성공률'] = np.where(test['총_시술_횟수'] > 0, test['총_임신_횟수'] / test['총_시술_횟수'], -1)

### 39. 총_생성_배아_수

해당 시술에서 생성된 배아의 총 개수

In [1657]:
value_counts_ratio(train, '총_생성_배아_수')


총_생성_배아_수별 임신_성공_여부 비율 및 갯수

                 0         1        0       1  Total
variable                                            
0.0       0.828147  0.171853  44180.0  9168.0  53348
1.0       0.905137  0.094863  13778.0  1444.0  15222
2.0       0.819906  0.180094  15734.0  3456.0  19190
3.0       0.758150  0.241850  15884.0  5067.0  20951
4.0       0.722700  0.277300  15619.0  5993.0  21612
5.0       0.694547  0.305453  14175.0  6234.0  20409
6.0       0.677012  0.322988  12608.0  6015.0  18623
7.0       0.661676  0.338324  10606.0  5423.0  16029
8.0       0.649974  0.350026   8852.0  4767.0  13619
9.0       0.645674  0.354326   7187.0  3944.0  11131
10.0      0.639563  0.360437   5797.0  3267.0   9064
11.0      0.633145  0.366855   4489.0  2601.0   7090
12.0      0.647552  0.352448   3704.0  2016.0   5720
13.0      0.633881  0.366119   2739.0  1582.0   4321
14.0      0.644048  0.355952   2164.0  1196.0   3360
15.0      0.642289  0.357711   1616.0   900.0   2516
16.0      0.6529

In [1658]:
# 결측값 개수 확인
print("결측값 개수:", train['총_생성_배아_수'].isnull().sum())

# 결측값을 0으로 채우기
train['총_생성_배아_수'].fillna(0, inplace=True)
test['총_생성_배아_수'].fillna(0, inplace=True)

# 결측값 개수 확인
print("결측값 개수:", train['총_생성_배아_수'].isnull().sum())

결측값 개수: 6290
결측값 개수: 0


### 40. 미세주입된_난자_수

정자 미세주입(ICSI)으로 처리된 난자의 수

In [1659]:
value_counts_ratio(train, '미세주입된_난자_수')


미세주입된_난자_수별 임신_성공_여부 비율 및 갯수

                 0         1        0        1   Total
variable                                              
0.0       0.749260  0.250740  95655.0  32011.0  127666
1.0       0.942681  0.057319   4276.0    260.0    4536
2.0       0.888783  0.111217   5594.0    700.0    6294
3.0       0.833418  0.166582   6594.0   1318.0    7912
4.0       0.789753  0.210247   7599.0   2023.0    9622
5.0       0.744609  0.255391   7907.0   2712.0   10619
6.0       0.723316  0.276684   8010.0   3064.0   11074
7.0       0.704831  0.295169   7338.0   3073.0   10411
8.0       0.693680  0.306320   6728.0   2971.0    9699
9.0       0.681734  0.318266   5867.0   2739.0    8606
10.0      0.665462  0.334538   5156.0   2592.0    7748
11.0      0.650375  0.349625   4245.0   2282.0    6527
12.0      0.660619  0.339381   3714.0   1908.0    5622
13.0      0.642012  0.357988   2986.0   1665.0    4651
14.0      0.661374  0.338626   2541.0   1301.0    3842
15.0      0.650893  0.349107   207

In [1660]:
# 결측값 개수 확인
print("결측값 개수:", train['미세주입된_난자_수'].isnull().sum())

# 결측값을 0으로 채우기
train['미세주입된_난자_수'].fillna(0, inplace=True)
test['미세주입된_난자_수'].fillna(0, inplace=True)

# 결측값 개수 확인
print("결측값 개수:", train['미세주입된_난자_수'].isnull().sum())

결측값 개수: 6290
결측값 개수: 0


### 41. 미세주입에서_생성된_배아_수

ICSI를 통해 생성된 배아의 수

In [1661]:
value_counts_ratio(train, '미세주입에서_생성된_배아_수')


미세주입에서_생성된_배아_수별 임신_성공_여부 비율 및 갯수

                 0         1        0        1   Total
variable                                              
0.0       0.756900  0.243100  99736.0  32033.0  131769
1.0       0.908532  0.091468   9009.0    907.0    9916
2.0       0.817908  0.182092  10331.0   2300.0   12631
3.0       0.758276  0.241724  10468.0   3337.0   13805
4.0       0.719729  0.280271   9992.0   3891.0   13883
5.0       0.689546  0.310454   8911.0   4012.0   12923
6.0       0.671874  0.328126   7785.0   3802.0   11587
7.0       0.659416  0.340584   6275.0   3241.0    9516
8.0       0.651130  0.348870   5071.0   2717.0    7788
9.0       0.641665  0.358335   3961.0   2212.0    6173
10.0      0.635817  0.364183   3064.0   1755.0    4819
11.0      0.624231  0.375769   2334.0   1405.0    3739
12.0      0.662865  0.337135   1976.0   1005.0    2981
13.0      0.642027  0.357973   1381.0    770.0    2151
14.0      0.646536  0.353464   1092.0    597.0    1689
15.0      0.653974  0.346026 

In [1662]:
# 결측값 개수 확인
print("결측값 개수:", train['미세주입에서_생성된_배아_수'].isnull().sum())

# 결측값을 0으로 채우기
train['미세주입에서_생성된_배아_수'].fillna(0, inplace=True)
test['미세주입에서_생성된_배아_수'].fillna(0, inplace=True)

# 결측값 개수 확인
print("결측값 개수:", train['미세주입에서_생성된_배아_수'].isnull().sum())

결측값 개수: 6290
결측값 개수: 0


### 42. 이식된_배아_수

이식된 배아의 총 수

In [1663]:
value_counts_ratio(train, '이식된_배아_수')


이식된_배아_수별 임신_성공_여부 비율 및 갯수

                 0         1      0      1   Total
variable                                          
0.0       0.999179  0.000821  36512     30   36542
1.0       0.686456  0.313544  64382  29407   93789
2.0       0.688911  0.311089  76361  34482  110843
3.0       0.831532  0.168468   7384   1496    8880


In [1664]:
# 결측값 개수 확인
print("결측값 개수:", train['이식된_배아_수'].isnull().sum())

# 결측값을 0으로 채우기
train['이식된_배아_수'].fillna(0, inplace=True)
test['이식된_배아_수'].fillna(0, inplace=True)

# 결측값 개수 확인
print("결측값 개수:", train['이식된_배아_수'].isnull().sum())

결측값 개수: 6290
결측값 개수: 0


### 43. 미세주입_배아_이식_수

ICSI 배아 중 이식된 배아의 수

In [1665]:
value_counts_ratio(train, '미세주입_배아_이식_수')


미세주입_배아_이식_수별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
0.0       0.776133  0.223867  107704  31066  138770
1.0       0.680559  0.319441   31629  14846   46475
2.0       0.687683  0.312317   41043  18640   59683
3.0       0.831643  0.168357    4263    863    5126


In [1666]:
# 결측값 개수 확인
print("결측값 개수:", train['미세주입_배아_이식_수'].isnull().sum())

# 결측값을 0으로 채우기
train['미세주입_배아_이식_수'].fillna(0, inplace=True)
test['미세주입_배아_이식_수'].fillna(0, inplace=True)

# 결측값 개수 확인
print("결측값 개수:", train['미세주입_배아_이식_수'].isnull().sum())

결측값 개수: 6290
결측값 개수: 0


### 44. 저장된_배아_수

환자를 위해 저장된 배아의 수

In [1667]:
value_counts_ratio(train, '저장된_배아_수')


저장된_배아_수별 임신_성공_여부 비율 및 갯수

                 0         1         0        1   Total
variable                                               
0.0       0.777878  0.222122  129799.0  37064.0  166863
1.0       0.654653  0.345347   13332.0   7033.0   20365
2.0       0.629509  0.370491   12514.0   7365.0   19879
3.0       0.630304  0.369696    8511.0   4992.0   13503
4.0       0.639706  0.360294    5916.0   3332.0    9248
5.0       0.650552  0.349448    3889.0   2089.0    5978
6.0       0.667477  0.332523    2746.0   1368.0    4114
7.0       0.705461  0.294539    1873.0    782.0    2655
8.0       0.714971  0.285029    1342.0    535.0    1877
9.0       0.740482  0.259518     953.0    334.0    1287
10.0      0.781065  0.218935     792.0    222.0    1014
11.0      0.828076  0.171924     525.0    109.0     634
12.0      0.841121  0.158879     450.0     85.0     535
13.0      0.911616  0.088384     361.0     35.0     396
14.0      0.936232  0.063768     323.0     22.0     345
15.0      0.929134 

In [1668]:
# 결측값 개수 확인
print("결측값 개수:", train['저장된_배아_수'].isnull().sum())

# 결측값을 0으로 채우기
train['저장된_배아_수'].fillna(0, inplace=True)
test['저장된_배아_수'].fillna(0, inplace=True)

# 결측값 개수 확인
print("결측값 개수:", train['저장된_배아_수'].isnull().sum())

결측값 개수: 6290
결측값 개수: 0


### 45. 미세주입_후_저장된_배아_수

ICSI 처리된 배아 중 저장된 배아의 수

In [1669]:
value_counts_ratio(train, '미세주입_후_저장된_배아_수')


미세주입_후_저장된_배아_수별 임신_성공_여부 비율 및 갯수

                 0         1         0        1   Total
variable                                               
0.0       0.755905  0.244095  154423.0  49866.0  204289
1.0       0.653485  0.346515    8064.0   4276.0   12340
2.0       0.624791  0.375209    7092.0   4259.0   11351
3.0       0.633647  0.366353    4708.0   2722.0    7430
4.0       0.645731  0.354269    3146.0   1726.0    4872
5.0       0.656188  0.343812    1983.0   1039.0    3022
6.0       0.673375  0.326625    1305.0    633.0    1938
7.0       0.741986  0.258014     949.0    330.0    1279
8.0       0.741419  0.258581     648.0    226.0     874
9.0       0.768233  0.231767     474.0    143.0     617
10.0      0.830835  0.169165     388.0     79.0     467
11.0      0.851724  0.148276     247.0     43.0     290
12.0      0.853282  0.146718     221.0     38.0     259
13.0      0.925000  0.075000     185.0     15.0     200
14.0      0.971429  0.028571     170.0      5.0     175
15.0      0.

In [1670]:
# 결측값 개수 확인
print("결측값 개수:", train['미세주입_후_저장된_배아_수'].isnull().sum())

# 결측값을 0으로 채우기
train['미세주입_후_저장된_배아_수'].fillna(0, inplace=True)
test['미세주입_후_저장된_배아_수'].fillna(0, inplace=True)

# 결측값 개수 확인
print("결측값 개수:", train['미세주입_후_저장된_배아_수'].isnull().sum())

결측값 개수: 6290
결측값 개수: 0


### 46. 해동된_배아_수

동결 해동된 배아의 수

In [1671]:
value_counts_ratio(train, '해동된_배아_수')


해동된_배아_수별 임신_성공_여부 비율 및 갯수

                 0         1         0        1   Total
variable                                               
0.0       0.732017  0.267983  153482.0  56188.0  209670
1.0       0.784757  0.215243   10245.0   2810.0   13055
2.0       0.761242  0.238758    9192.0   2883.0   12075
3.0       0.794957  0.205043    4288.0   1106.0    5394
4.0       0.778679  0.221321    2688.0    764.0    3452
5.0       0.766133  0.233867    1484.0    453.0    1937
6.0       0.752502  0.247498    1128.0    371.0    1499
7.0       0.759463  0.240537     622.0    197.0     819
8.0       0.699068  0.300932     525.0    226.0     751
9.0       0.672330  0.327670     277.0    135.0     412
10.0      0.741279  0.258721     255.0     89.0     344
11.0      0.756410  0.243590     118.0     38.0     156
12.0      0.673611  0.326389      97.0     47.0     144
13.0      0.679012  0.320988      55.0     26.0      81
14.0      0.720000  0.280000      54.0     21.0      75
15.0      0.714286 

In [1672]:
# 결측값 개수 확인
print("결측값 개수:", train['해동된_배아_수'].isnull().sum())

# 결측값을 0으로 채우기
train['해동된_배아_수'].fillna(0, inplace=True)
test['해동된_배아_수'].fillna(0, inplace=True)

# 결측값 개수 확인
print("결측값 개수:", train['해동된_배아_수'].isnull().sum())

결측값 개수: 6290
결측값 개수: 0


### 47. 해동_난자_수

동결 해동된 난자의 수

In [1673]:
value_counts_ratio(train, '해동_난자_수')


해동_난자_수별 임신_성공_여부 비율 및 갯수

                 0         1         0        1   Total
variable                                               
0.0       0.738167  0.261833  183515.0  65094.0  248609
1.0       0.923077  0.076923      24.0      2.0      26
2.0       0.907692  0.092308      59.0      6.0      65
3.0       0.907692  0.092308      59.0      6.0      65
4.0       0.892308  0.107692     116.0     14.0     130
5.0       0.859813  0.140187      92.0     15.0     107
6.0       0.801170  0.198830     137.0     34.0     171
7.0       0.713287  0.286713     102.0     41.0     143
8.0       0.703196  0.296804     154.0     65.0     219
9.0       0.780952  0.219048      82.0     23.0     105
10.0      0.769231  0.230769      70.0     21.0      91
11.0      0.761194  0.238806      51.0     16.0      67
12.0      0.657143  0.342857      46.0     24.0      70
13.0      0.812500  0.187500      26.0      6.0      32
14.0      0.613636  0.386364      27.0     17.0      44
15.0      0.594595  

In [1674]:
# 결측값 개수 확인
print("결측값 개수:", train['해동_난자_수'].isnull().sum())

# 결측값을 0으로 채우기
train['해동_난자_수'].fillna(0, inplace=True)
test['해동_난자_수'].fillna(0, inplace=True)

# 결측값 개수 확인
print("결측값 개수:", train['해동_난자_수'].isnull().sum())

결측값 개수: 6290
결측값 개수: 0


### 48. 수집된_신선_난자_수

신선 난자가 수집된 수

In [1675]:
value_counts_ratio(train, '수집된_신선_난자_수')


수집된_신선_난자_수별 임신_성공_여부 비율 및 갯수

                 0         1        0        1  Total
variable                                             
0.0       0.759528  0.240472  40896.0  12948.0  53844
1.0       0.948279  0.051721   4602.0    251.0   4853
2.0       0.909020  0.090980   5855.0    586.0   6441
3.0       0.867346  0.132654   7068.0   1081.0   8149
4.0       0.824000  0.176000   8240.0   1760.0  10000
5.0       0.790379  0.209621   9102.0   2414.0  11516
6.0       0.773797  0.226203   9763.0   2854.0  12617
7.0       0.746945  0.253055   9779.0   3313.0  13092
8.0       0.725173  0.274827   9668.0   3664.0  13332
9.0       0.718030  0.281970   9287.0   3647.0  12934
10.0      0.697813  0.302187   8902.0   3855.0  12757
11.0      0.682752  0.317248   8178.0   3800.0  11978
12.0      0.671998  0.328002   7353.0   3589.0  10942
13.0      0.664711  0.335289   6570.0   3314.0   9884
14.0      0.668292  0.331708   6032.0   2994.0   9026
15.0      0.662559  0.337441   5162.0   2629.0   7

In [1676]:
# '수집된_신선_난자_수' 변수의 값이 10 에서 24 이하인 경우 더미 변수로 1, 그렇지 않으면 0 부여
train['수집된_신선난자수_적정'] = train['수집된_신선_난자_수'].apply(lambda x: 1 if 10 <= x <= 24 else 0)
test['수집된_신선난자수_적정'] = test['수집된_신선_난자_수'].apply(lambda x: 1 if 10 <= x <= 24 else 0)

In [1677]:
# 결측값 개수 확인
print("결측값 개수:", train['수집된_신선_난자_수'].isnull().sum())

# 결측값을 0으로 채우기
train['수집된_신선_난자_수'].fillna(0, inplace=True)
test['수집된_신선_난자_수'].fillna(0, inplace=True)

# 결측값 개수 확인
print("결측값 개수:", train['수집된_신선_난자_수'].isnull().sum())

결측값 개수: 6290
결측값 개수: 0


### 49. 저장된_신선_난자_수

저장된 신선 난자의 수

In [1678]:
value_counts_ratio(train, '저장된_신선_난자_수')


저장된_신선_난자_수별 임신_성공_여부 비율 및 갯수

                 0         1         0        1   Total
variable                                               
0.0       0.735751  0.264249  182016.0  65372.0  247388
1.0       0.972414  0.027586     141.0      4.0     145
2.0       0.989418  0.010582     187.0      2.0     189
3.0       0.985075  0.014925     198.0      3.0     201
4.0       0.980392  0.019608     200.0      4.0     204
5.0       0.972350  0.027650     211.0      6.0     217
6.0       0.981308  0.018692     210.0      4.0     214
7.0       0.977778  0.022222     176.0      4.0     180
8.0       0.994536  0.005464     182.0      1.0     183
9.0       0.979592  0.020408     144.0      3.0     147
10.0      0.993197  0.006803     146.0      1.0     147
11.0      0.967213  0.032787     118.0      4.0     122
12.0      0.982609  0.017391     113.0      2.0     115
13.0      1.000000  0.000000      86.0      0.0      86
14.0      0.977273  0.022727      86.0      2.0      88
15.0      0.9873

In [1679]:
# '저장된_신선_난자_수' 변수의 값이 12 이상인 경우 더미 변수로 1 부여
train['저장된_신선난자수_초괴'] = train['저장된_신선_난자_수'].apply(lambda x: 1 if x >= 12 else 0)
test['저장된_신선난자수_초괴'] = test['저장된_신선_난자_수'].apply(lambda x: 1 if x >= 12 else 0)

In [1680]:
# '저장된_신선_난자_수'와 '수집된_신선_난자_수'의 비율을 계산하여 새로운 파생 변수 생성
def calculate_ratio(row):
    if row['수집된_신선_난자_수'] == 0:
        return -1
    else:
        return round(row['저장된_신선_난자_수'] / row['수집된_신선_난자_수'], 1)

train['저장된_신선_난자_비율'] = train.apply(calculate_ratio, axis=1)
test['저장된_신선_난자_비율'] = test.apply(calculate_ratio, axis=1)

In [1681]:
pd.set_option('display.max_rows', 300)

In [1682]:
value_counts_ratio(train, '저장된_신선_난자_비율')


저장된_신선_난자_비율별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
-1.0      0.771194  0.228806   46375  13759   60134
 0.0      0.729138  0.270862  141121  52424  193545
 0.1      0.714286  0.285714      10      4      14
 0.2      0.946429  0.053571      53      3      56
 0.3      0.956522  0.043478     110      5     115
 0.4      0.965278  0.034722     139      5     144
 0.5      0.965649  0.034351     253      9     262
 0.6      0.963158  0.036842     183      7     190
 0.7      0.989967  0.010033     296      3     299
 0.8      0.989035  0.010965     451      5     456
 0.9      0.997613  0.002387     418      1     419
 1.0      0.998592  0.001408     709      1     710


In [1683]:
# 결측값 개수 확인
print("결측값 개수:", train['저장된_신선_난자_수'].isnull().sum())

# 결측값을 0으로 채우기
train['저장된_신선_난자_수'].fillna(0, inplace=True)
test['저장된_신선_난자_수'].fillna(0, inplace=True)

# 결측값 개수 확인
print("결측값 개수:", train['저장된_신선_난자_수'].isnull().sum())

결측값 개수: 6290
결측값 개수: 0


### 50. 혼합된_난자_수

정자와 혼합된 총 난자 수

In [1684]:
value_counts_ratio(train, '혼합된_난자_수')


혼합된_난자_수별 임신_성공_여부 비율 및 갯수

                 0         1        0       1  Total
variable                                            
0.0       0.803790  0.196210  37496.0  9153.0  46649
1.0       0.939370  0.060630   6058.0   391.0   6449
2.0       0.892197  0.107803   7970.0   963.0   8933
3.0       0.837625  0.162375   9270.0  1797.0  11067
4.0       0.794531  0.205469  10808.0  2795.0  13603
5.0       0.750419  0.249581  11639.0  3871.0  15510
6.0       0.735992  0.264008  12032.0  4316.0  16348
7.0       0.714706  0.285294  11416.0  4557.0  15973
8.0       0.698027  0.301973  10647.0  4606.0  15253
9.0       0.689592  0.310408   9726.0  4378.0  14104
10.0      0.672614  0.327386   8746.0  4257.0  13003
11.0      0.661431  0.338569   7664.0  3923.0  11587
12.0      0.661170  0.338830   6736.0  3452.0  10188
13.0      0.646627  0.353373   5636.0  3080.0   8716
14.0      0.659124  0.340876   4981.0  2576.0   7557
15.0      0.656989  0.343011   4277.0  2233.0   6510
16.0      0.64786

In [1685]:
# 결측값 개수 확인
print("결측값 개수:", train['혼합된_난자_수'].isnull().sum())

# 결측값을 0으로 채우기
train['혼합된_난자_수'].fillna(0, inplace=True)
test['혼합된_난자_수'].fillna(0, inplace=True)

# 결측값 개수 확인
print("결측값 개수:", train['혼합된_난자_수'].isnull().sum())

결측값 개수: 6290
결측값 개수: 0


### 51. 파트너_정자와_혼합된_난자_수

파트너 정자와 혼합된 난자의 수

In [1686]:
value_counts_ratio(train, '파트너_정자와_혼합된_난자_수')


파트너_정자와_혼합된_난자_수별 임신_성공_여부 비율 및 갯수

                 0         1        0        1  Total
variable                                             
0.0       0.782549  0.217451  49238.0  13682.0  62920
1.0       0.939346  0.060654   5374.0    347.0   5721
2.0       0.892287  0.107713   7207.0    870.0   8077
3.0       0.837771  0.162229   8464.0   1639.0  10103
4.0       0.796633  0.203367   9891.0   2525.0  12416
5.0       0.751072  0.248928  10687.0   3542.0  14229
6.0       0.737459  0.262541  11070.0   3941.0  15011
7.0       0.717054  0.282946  10545.0   4161.0  14706
8.0       0.698579  0.301421   9829.0   4241.0  14070
9.0       0.690976  0.309024   9020.0   4034.0  13054
10.0      0.674378  0.325622   8104.0   3913.0  12017
11.0      0.660474  0.339526   7075.0   3637.0  10712
12.0      0.661418  0.338582   6259.0   3204.0   9463
13.0      0.646854  0.353146   5202.0   2840.0   8042
14.0      0.659744  0.340256   4638.0   2392.0   7030
15.0      0.656934  0.343066   3960.0   2068.

In [1687]:
# 결측값 개수 확인
print("결측값 개수:", train['파트너_정자와_혼합된_난자_수'].isnull().sum())

# 결측값을 0으로 채우기
train['파트너_정자와_혼합된_난자_수'].fillna(0, inplace=True)
test['파트너_정자와_혼합된_난자_수'].fillna(0, inplace=True)

# 결측값 개수 확인
print("결측값 개수:", train['파트너_정자와_혼합된_난자_수'].isnull().sum())

결측값 개수: 6290
결측값 개수: 0


### 52. 기증자_정자와_혼합된_난자_수

기증자 정자와 혼합된 난자의 수

In [1688]:
value_counts_ratio(train, '기증자_정자와_혼합된_난자_수')


기증자_정자와_혼합된_난자_수별 임신_성공_여부 비율 및 갯수

                 0         1         0        1   Total
variable                                               
0.0       0.739554  0.260446  173455.0  61085.0  234540
1.0       0.937669  0.062331     692.0     46.0     738
2.0       0.887486  0.112514     773.0     98.0     871
3.0       0.835700  0.164300     824.0    162.0     986
4.0       0.770574  0.229426     927.0    276.0    1203
5.0       0.738497  0.261503     963.0    341.0    1304
6.0       0.715959  0.284041     978.0    388.0    1366
7.0       0.691083  0.308917     868.0    388.0    1256
8.0       0.692110  0.307890     807.0    359.0    1166
9.0       0.671952  0.328048     678.0    331.0    1009
10.0      0.647752  0.352248     605.0    329.0     934
11.0      0.667892  0.332108     545.0    271.0     816
12.0      0.653153  0.346847     435.0    231.0     666
13.0      0.636060  0.363940     381.0    218.0     599
14.0      0.645702  0.354298     308.0    169.0     477
15.0      0

In [1689]:
summarize_group(train, ['혼합된_난자_수','기증자_정자와_혼합된_난자_수', '파트너_정자와_혼합된_난자_수'])

Grouped by: 혼합된_난자_수, 기증자_정자와_혼합된_난자_수, 파트너_정자와_혼합된_난자_수

                  group  '임신_성공_여부' count     ratio  Total
0       (0.0, 0.0, 0.0)              9153  0.196210  46649
1       (1.0, 0.0, 1.0)               346  0.060553   5714
2       (1.0, 1.0, 0.0)                45  0.061224    735
3       (2.0, 0.0, 2.0)               866  0.107364   8066
4       (2.0, 1.0, 1.0)                 0  0.000000      1
5       (2.0, 2.0, 0.0)                97  0.112009    866
6       (3.0, 0.0, 3.0)              1636  0.162125  10091
7       (3.0, 2.0, 1.0)                 0  0.000000      1
8       (3.0, 3.0, 0.0)               161  0.165128    975
9       (4.0, 0.0, 0.0)                 0  0.000000      5
10      (4.0, 0.0, 4.0)              2522  0.203354  12402
11      (4.0, 1.0, 3.0)                 1  0.500000      2
12      (4.0, 3.0, 1.0)                 0  0.000000      1
13      (4.0, 4.0, 0.0)               272  0.227997   1193
14      (5.0, 0.0, 0.0)                 0  0.000000      

In [1690]:
# '기증자_정자와_혼합된_난자_수'와 '혼합된_난자_수'의 비율을 계산하여 새로운 파생 변수 생성
def calculate_donor_ratio(row):
    if row['혼합된_난자_수'] == 0:
        return -1
    else:
        return round(row['기증자_정자와_혼합된_난자_수'] / row['혼합된_난자_수'], 1)

# '파트너_정자와_혼합된_난자_수'와 '혼합된_난자_수'의 비율을 계산하여 새로운 파생 변수 생성
def calculate_partner_ratio(row):
    if row['혼합된_난자_수'] == 0:
        return -1
    else:
        return round(row['파트너_정자와_혼합된_난자_수'] / row['혼합된_난자_수'], 1)

train['기증자_정자_비율'] = train.apply(calculate_donor_ratio, axis=1)
test['기증자_정자_비율'] = test.apply(calculate_donor_ratio, axis=1)

train['파트너_정자_비율'] = train.apply(calculate_partner_ratio, axis=1)
test['파트너_정자_비율'] = test.apply(calculate_partner_ratio, axis=1)

In [1691]:
value_counts_ratio(train, '기증자_정자_비율')


기증자_정자_비율별 임신_성공_여부 비율 및 갯수

                 0         1         0        1   Total
variable                                               
-1.0      0.811783  0.188217   42975.0   9964.0   52939
 0.0      0.723606  0.276394  135959.0  51932.0  187891
 0.1      0.000000  1.000000       0.0      1.0       1
 0.2      0.500000  0.500000       3.0      3.0       6
 0.3      0.714286  0.285714       5.0      2.0       7
 0.4      0.833333  0.166667      10.0      2.0      12
 0.5      0.681818  0.318182      30.0     14.0      44
 0.6      0.823529  0.176471      14.0      3.0      17
 0.7      0.571429  0.428571       8.0      6.0      14
 0.8      0.583333  0.416667       7.0      5.0      12
 0.9      1.000000  0.000000       4.0      0.0       4
 1.0      0.721115  0.278885   11103.0   4294.0   15397


In [1692]:
value_counts_ratio(train, '파트너_정자_비율')


파트너_정자_비율별 임신_성공_여부 비율 및 갯수

                 0         1         0        1   Total
variable                                               
-1.0      0.811783  0.188217   42975.0   9964.0   52939
 0.0      0.721652  0.278348   11742.0   4529.0   16271
 0.1      1.000000  0.000000       4.0      0.0       4
 0.2      0.583333  0.416667       7.0      5.0      12
 0.3      0.571429  0.428571       8.0      6.0      14
 0.4      0.823529  0.176471      14.0      3.0      17
 0.5      0.681818  0.318182      30.0     14.0      44
 0.6      0.833333  0.166667      10.0      2.0      12
 0.7      0.714286  0.285714       5.0      2.0       7
 0.8      0.500000  0.500000       3.0      3.0       6
 0.9      0.000000  1.000000       0.0      1.0       1
 1.0      0.723571  0.276429  135320.0  51697.0  187017


In [1693]:
# 결측값 개수 확인
print("결측값 개수:", train['기증자_정자와_혼합된_난자_수'].isnull().sum())

# 결측값을 0으로 채우기
train['기증자_정자와_혼합된_난자_수'].fillna(0, inplace=True)
test['기증자_정자와_혼합된_난자_수'].fillna(0, inplace=True)

# 결측값 개수 확인
print("결측값 개수:", train['기증자_정자와_혼합된_난자_수'].isnull().sum())

결측값 개수: 6290
결측값 개수: 0


### 53. 난자_출처

난자 출처 정보

In [1694]:
value_counts_ratio(train, '난자_출처')


난자_출처별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
기증 제공     0.684571  0.315429   10795   4974   15769
본인 제공     0.742019  0.257981  173844  60441  234285
알 수 없음    0.871065  0.128935    5479    811    6290


In [1695]:
test['난자_출처'].value_counts()

본인 제공     82397
기증 제공      5494
알 수 없음     2176
Name: 난자_출처, dtype: int64

In [1696]:
# 원핫인코딩
train = pd.get_dummies(train, columns=['난자_출처'], prefix='난자_출처')
test = pd.get_dummies(test, columns=['난자_출처'], prefix='난자_출처')

In [1697]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256344 entries, 0 to 256343
Data columns (total 87 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   ID                     256344 non-null  object 
 1   시술_당시_나이               256344 non-null  object 
 2   임신_시도_또는_마지막_임신_경과_연수  256344 non-null  float64
 3   배란_자극_여부               256344 non-null  int64  
 4   단일_배아_이식_여부            256344 non-null  float64
 5   착상_전_유전_검사_사용_여부       256344 non-null  float64
 6   착상_전_유전_진단_사용_여부       256344 non-null  float64
 7   남성_주_불임_원인             256344 non-null  int64  
 8   남성_부_불임_원인             256344 non-null  int64  
 9   여성_주_불임_원인             256344 non-null  int64  
 10  여성_부_불임_원인             256344 non-null  int64  
 11  부부_주_불임_원인             256344 non-null  int64  
 12  부부_부_불임_원인             256344 non-null  int64  
 13  불명확_불임_원인              256344 non-null  int64  
 14  불임_원인_-_난관_질환          256344 non-nu

In [1698]:
# '난자_출처_알 수 없음' 열을 제거
train = train.drop(columns=['난자_출처_알 수 없음'])
test = test.drop(columns=['난자_출처_알 수 없음'])

### 54. 정자_출처

정자 출처 정보

In [1699]:
value_counts_ratio(train, '정자_출처')


정자_출처별 임신_성공_여부 비율 및 갯수

                    0         1         0        1   Total
variable                                                  
기증 제공        0.756247  0.243753   20430.0   6585.0   27015
미할당          0.893443  0.106557     109.0     13.0     122
배우자 및 기증 제공  1.000000  0.000000      14.0      0.0      14
배우자 제공       0.739835  0.260165  169565.0  59628.0  229193


In [1700]:
test['정자_출처'].value_counts()

배우자 제공         80569
기증 제공           9460
미할당               33
배우자 및 기증 제공        5
Name: 정자_출처, dtype: int64

In [1701]:
# '배우자 제공'과 '기증 제공' 값을 취하는 더미 변수 생성
def create_dummy_variables(df):
    df['정자_출처_배우자_제공'] = df['정자_출처'].apply(lambda x: 1 if x in ['배우자 제공', '배우자 및 기증 제공'] else 0)
    df['정자_출처_기증_제공'] = df['정자_출처'].apply(lambda x: 1 if x in ['기증 제공', '배우자 및 기증 제공'] else 0)
    return df

train = create_dummy_variables(train)
test = create_dummy_variables(test)

In [1702]:
train = train.drop(columns=['정자_출처'])
test = test.drop(columns=['정자_출처'])

### 55. 난자_기증자_나이

난자 기증자의 등록 당시 나이

In [1703]:
value_counts_ratio(train, '난자_기증자_나이')


난자_기증자_나이별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
만20세 이하   0.738095  0.261905     217     77     294
만21-25세   0.670094  0.329906    1564    770    2334
만26-30세   0.651728  0.348272    3243   1733    4976
만31-35세   0.694785  0.305215    4423   1943    6366
알 수 없음    0.745422  0.254578  180671  61703  242374


In [1704]:
# 결측값 개수 확인
print("결측값 개수:", train['난자_기증자_나이'].isnull().sum())

결측값 개수: 0


### 56. 정자_기증자_나이

정자 기증자의 등록 당시 나이

In [1705]:
value_counts_ratio(train, '정자_기증자_나이')


정자_기증자_나이별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
만20세 이하   0.790066  0.209934     843    224    1067
만21-25세   0.758602  0.241398    4299   1368    5667
만26-30세   0.751878  0.248122    3803   1255    5058
만31-35세   0.751374  0.248626    3690   1221    4911
만36-40세   0.754260  0.245740    3984   1298    5282
만41-45세   0.756953  0.243047    2912    935    3847
알 수 없음    0.740035  0.259965  170587  59925  230512


In [1706]:
# 결측값 개수 확인
print("결측값 개수:", train['정자_기증자_나이'].isnull().sum())

결측값 개수: 0


### 57. 동결_배아_사용_여부

동결 배아가 사용되었는지 여부

In [1707]:
value_counts_ratio(train, '동결_배아_사용_여부')


동결_배아_사용_여부별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
0.0       0.732238  0.267762  153718  56211  209929
1.0       0.770617  0.229383   30921   9204   40125


In [1708]:
# 결측값 개수 확인
print("결측값 개수:", train['동결_배아_사용_여부'].isnull().sum())

# 결측값을 0으로 채우기
train['동결_배아_사용_여부'].fillna(0, inplace=True)
test['동결_배아_사용_여부'].fillna(0, inplace=True)

# 결측값 개수 확인
print("결측값 개수:", train['동결_배아_사용_여부'].isnull().sum())

결측값 개수: 6290
결측값 개수: 0


### 58. 신선_배아_사용_여부

신선 배아가 사용되었는지 여부

In [1709]:
value_counts_ratio(train, '신선_배아_사용_여부')


신선_배아_사용_여부별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
0.0       0.770433  0.229567   30758   9165   39923
1.0       0.732310  0.267690  153881  56250  210131


In [1710]:
# 결측값 개수 확인
print("결측값 개수:", train['신선_배아_사용_여부'].isnull().sum())

# 결측값을 0으로 채우기
train['신선_배아_사용_여부'].fillna(0, inplace=True)
test['신선_배아_사용_여부'].fillna(0, inplace=True)

# 결측값 개수 확인
print("결측값 개수:", train['신선_배아_사용_여부'].isnull().sum())

결측값 개수: 6290
결측값 개수: 0


### 59. 기증_배아_사용_여부

기증 배아가 사용되었는지 여부

In [1711]:
value_counts_ratio(train, '기증_배아_사용_여부')


기증_배아_사용_여부별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
0.0       0.739051  0.260949  182986  64610  247596
1.0       0.672498  0.327502    1653    805    2458


In [1712]:
summarize_group(train, ['신선_배아_사용_여부', '동결_배아_사용_여부', '기증_배아_사용_여부'])

Grouped by: 신선_배아_사용_여부, 동결_배아_사용_여부, 기증_배아_사용_여부

             group  '임신_성공_여부' count     ratio   Total
0  (0.0, 0.0, 0.0)                 1  0.333333       3
1  (0.0, 1.0, 0.0)              8916  0.229009   38933
2  (0.0, 1.0, 1.0)               248  0.251266     987
3  (1.0, 0.0, 0.0)             55653  0.266978  208455
4  (1.0, 0.0, 1.0)               557  0.378654    1471
5  (1.0, 1.0, 0.0)                40  0.195122     205


In [1713]:
# 결측값 개수 확인
print("결측값 개수:", train['기증_배아_사용_여부'].isnull().sum())

# 결측값을 0으로 채우기
train['기증_배아_사용_여부'].fillna(0, inplace=True)
test['기증_배아_사용_여부'].fillna(0, inplace=True)

# 결측값 개수 확인
print("결측값 개수:", train['기증_배아_사용_여부'].isnull().sum())

결측값 개수: 6290
결측값 개수: 0


### 60. 대리모_여부

환자가 대리모로 활동했는지 여부

In [1714]:
value_counts_ratio(train, '대리모_여부')


대리모_여부별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
0.0       0.738511  0.261489  183893  65112  249005
1.0       0.711153  0.288847     746    303    1049


In [1715]:
# 결측값 개수 확인
print("결측값 개수:", train['대리모_여부'].isnull().sum())

# 결측값을 0으로 채우기
train['대리모_여부'].fillna(0, inplace=True)
test['대리모_여부'].fillna(0, inplace=True)

# 결측값 개수 확인
print("결측값 개수:", train['대리모_여부'].isnull().sum())

결측값 개수: 6290
결측값 개수: 0


### 61. PGD_시술_여부

해당 주기가 PGD 치료로 분류되는지 여부

In [1716]:
value_counts_ratio(train, 'PGD_시술_여부')


PGD_시술_여부별 임신_성공_여부 비율 및 갯수

                 0         1     0    1  Total
variable                                      
1.0       0.720514  0.279486  1570  609   2179


In [1717]:
train['PGD_시술_여부'].isnull().sum()

254165

In [1718]:
# 결측값 개수 확인
print("결측값 개수:", train['PGD_시술_여부'].isnull().sum())

# 결측값을 0으로 채우기
train['PGD_시술_여부'].fillna(0, inplace=True)
test['PGD_시술_여부'].fillna(0, inplace=True)

# 결측값 개수 확인
print("결측값 개수:", train['PGD_시술_여부'].isnull().sum())

결측값 개수: 254165
결측값 개수: 0


### 62. PGS_시술_여부

해당 주기가 PGS 치료로 분류되는지 여부

In [1719]:
value_counts_ratio(train, 'PGS_시술_여부')


PGS_시술_여부별 임신_성공_여부 비율 및 갯수

                 0         1     0    1  Total
variable                                      
1.0       0.793675  0.206325  1531  398   1929


In [1720]:
# 결측값 개수 확인
print("결측값 개수:", train['PGS_시술_여부'].isnull().sum())

# 결측값을 0으로 채우기
train['PGS_시술_여부'].fillna(0, inplace=True)
test['PGS_시술_여부'].fillna(0, inplace=True)

# 결측값 개수 확인
print("결측값 개수:", train['PGS_시술_여부'].isnull().sum())

결측값 개수: 254415
결측값 개수: 0


In [1721]:
summarize_group(train, ['PGD_시술_여부', 'PGS_시술_여부'])

Grouped by: PGD_시술_여부, PGS_시술_여부

        group  '임신_성공_여부' count     ratio   Total
0  (0.0, 0.0)             65219  0.258563  252236
1  (0.0, 1.0)               398  0.206325    1929
2  (1.0, 0.0)               609  0.279486    2179


### 63. 난자_채취_경과일

첫 시점으로부터 난자 채취까지 경과한 일수

In [1722]:
value_counts_ratio(train, '난자_채취_경과일')


난자_채취_경과일별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
0.0       0.736184  0.263816  146396  52462  198858


In [1723]:
print("결측값 개수:", train['난자_채취_경과일'].isnull().sum())

train['난자_채취_경과일'].fillna(-1, inplace=True)
test['난자_채취_경과일'].fillna(-1, inplace=True)

print("결측값 개수:", train['난자_채취_경과일'].isnull().sum())

결측값 개수: 57486
결측값 개수: 0


In [1724]:
# '난자_채취_경과일' 변수의 값이 0 인 경우 더미 변수로 1 부여
train['난자채취_적정기간'] = train['난자_채취_경과일'].apply(lambda x: 1 if x == 0 else 0)
test['난자채취_적정기간'] = test['난자_채취_경과일'].apply(lambda x: 1 if x == 0 else 0)

### 64. 난자_해동_경과일

첫 시점으로부터 난자 해동까지 경과한 일수

In [1725]:
value_counts_ratio(train, '난자_해동_경과일')


난자_해동_경과일별 임신_성공_여부 비율 및 갯수

                 0         1       0      1  Total
variable                                          
0.0       0.777545  0.222455  1115.0  319.0   1434
1.0       1.000000  0.000000     2.0    0.0      2


In [1726]:
train['난자_해동_경과일'].isnull().sum()

254908

In [1727]:
print("결측값 개수:", train['난자_해동_경과일'].isnull().sum())

train['난자_해동_경과일'].fillna(-1, inplace=True)
test['난자_해동_경과일'].fillna(-1, inplace=True)

print("결측값 개수:", train['난자_해동_경과일'].isnull().sum())

결측값 개수: 254908
결측값 개수: 0


In [1728]:
# '난자_혼합_경과일' 변수의 값이 0 인 경우 더미 변수로 1 부여
train['난자해동_적정기간'] = train['난자_해동_경과일'].apply(lambda x: 1 if x == 0 else 0)
test['난자해동_적정기간'] = test['난자_해동_경과일'].apply(lambda x: 1 if x == 0 else 0)

### 65. 난자_혼합_경과일

첫 시점으로부터 난자와 정자 혼합까지 경과한 일수

In [1729]:
value_counts_ratio(train, '난자_혼합_경과일')


난자_혼합_경과일별 임신_성공_여부 비율 및 갯수

                 0         1         0        1   Total
variable                                               
0.0       0.722924  0.277076  145970.0  55946.0  201916
1.0       0.846311  0.153689     413.0     75.0     488
2.0       0.754902  0.245098      77.0     25.0     102
3.0       0.720588  0.279412      49.0     19.0      68
4.0       0.600000  0.400000       3.0      2.0       5
5.0       0.791667  0.208333      19.0      5.0      24
6.0       1.000000  0.000000       8.0      0.0       8
7.0       1.000000  0.000000       1.0      0.0       1


In [1730]:
train['난자_혼합_경과일'].isnull().sum()

53732

In [1731]:
print("결측값 개수:", train['난자_혼합_경과일'].isnull().sum())

train['난자_혼합_경과일'].fillna(-1, inplace=True)
test['난자_혼합_경과일'].fillna(-1, inplace=True)

print("결측값 개수:", train['난자_혼합_경과일'].isnull().sum())

결측값 개수: 53732
결측값 개수: 0


In [1732]:
value_counts_ratio(train, '난자_혼합_경과일')


난자_혼합_경과일별 임신_성공_여부 비율 및 갯수

                 0         1         0        1   Total
variable                                               
-1.0      0.811025  0.188975   43578.0  10154.0   53732
 0.0      0.722924  0.277076  145970.0  55946.0  201916
 1.0      0.846311  0.153689     413.0     75.0     488
 2.0      0.754902  0.245098      77.0     25.0     102
 3.0      0.720588  0.279412      49.0     19.0      68
 4.0      0.600000  0.400000       3.0      2.0       5
 5.0      0.791667  0.208333      19.0      5.0      24
 6.0      1.000000  0.000000       8.0      0.0       8
 7.0      1.000000  0.000000       1.0      0.0       1


In [1733]:
# '난자_혼합_경과일' 변수의 값이 0인 경우 더미 변수로 1, 값이 1이상인 경우 0으로 부여
train['난자혼합_적정기간'] = train['난자_혼합_경과일'].apply(lambda x: 1 if x == 0 else 0)
test['난자혼합_적정기간'] = test['난자_혼합_경과일'].apply(lambda x: 1 if x == 0 else 0)

### 66. 배아_이식_경과일

첫 시점으로부터 배아 이식까지 경과한 일수

In [1734]:
value_counts_ratio(train, '배아_이식_경과일')


배아_이식_경과일별 임신_성공_여부 비율 및 갯수

                 0         1      0      1  Total
variable                                         
0.0       0.748986  0.251014  18652   6251  24903
1.0       0.812985  0.187015   4921   1132   6053
2.0       0.787531  0.212469  27625   7453  35078
3.0       0.741243  0.258757  42935  14988  57923
4.0       0.655639  0.344361   2953   1551   4504
5.0       0.595546  0.404454  48512  32946  81458
6.0       0.699964  0.300036   1941    832   2773
7.0       0.588889  0.411111     53     37     90


In [1735]:
print("결측값 개수:", train['배아_이식_경과일'].isnull().sum())

train['배아_이식_경과일'].fillna(-1, inplace=True)
test['배아_이식_경과일'].fillna(-1, inplace=True)

print("결측값 개수:", train['배아_이식_경과일'].isnull().sum())

결측값 개수: 43562
결측값 개수: 0


In [1736]:
# '배아_이식_경과일' 변수의 값이 4에서 7 이하인 경우 더미 변수로 1, 그렇지 않으면 0 부여
train['배아이식_적정기간'] = train['배아_이식_경과일'].apply(lambda x: 1 if 4 <= x <= 7 else 0)
test['배아이식_적정기간'] = test['배아_이식_경과일'].apply(lambda x: 1 if 4 <= x <= 7 else 0)

### 67. 배아_해동_경과일

첫 시점으로부터 배아 해동까지 경과한 일수

In [1737]:
value_counts_ratio(train, '배아_해동_경과일')


배아_해동_경과일별 임신_성공_여부 비율 및 갯수

                 0         1        0       1  Total
variable                                            
0.0       0.770352  0.229648  30660.0  9140.0  39800
1.0       0.837838  0.162162     31.0     6.0     37
2.0       0.840659  0.159341    153.0    29.0    182
3.0       0.890244  0.109756    146.0    18.0    164
4.0       1.000000  0.000000     12.0     0.0     12
5.0       0.785714  0.214286    110.0    30.0    140
6.0       0.833333  0.166667     25.0     5.0     30
7.0       1.000000  0.000000      3.0     0.0      3


In [1738]:
test['배아_해동_경과일'].value_counts()

0.0    13751
3.0       59
2.0       57
5.0       55
1.0       17
6.0        7
4.0        4
Name: 배아_해동_경과일, dtype: int64

In [1739]:
print("결측값 개수:", train['배아_해동_경과일'].isnull().sum())

train['배아_해동_경과일'].fillna(-1, inplace=True)
test['배아_해동_경과일'].fillna(-1, inplace=True)

print("결측값 개수:", train['배아_해동_경과일'].isnull().sum())

결측값 개수: 215976
결측값 개수: 0


In [1740]:
value_counts_ratio(train, '배아_해동_경과일')


배아_해동_경과일별 임신_성공_여부 비율 및 갯수

                 0         1         0        1   Total
variable                                               
-1.0      0.736091  0.263909  158978.0  56998.0  215976
 0.0      0.770352  0.229648   30660.0   9140.0   39800
 1.0      0.837838  0.162162      31.0      6.0      37
 2.0      0.840659  0.159341     153.0     29.0     182
 3.0      0.890244  0.109756     146.0     18.0     164
 4.0      1.000000  0.000000      12.0      0.0      12
 5.0      0.785714  0.214286     110.0     30.0     140
 6.0      0.833333  0.166667      25.0      5.0      30
 7.0      1.000000  0.000000       3.0      0.0       3


In [1741]:
# '배아_해동_경과일' 변수의 값이 0인 경우 더미 변수로 1 부여
train['배아해동_적정기간'] = train['배아_해동_경과일'].apply(lambda x: 1 if x == 0 else 0)
test['배아해동_적정기간'] = test['배아_해동_경과일'].apply(lambda x: 1 if x == 0 else 0)

In [1742]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256344 entries, 0 to 256343
Data columns (total 92 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   ID                     256344 non-null  object 
 1   시술_당시_나이               256344 non-null  object 
 2   임신_시도_또는_마지막_임신_경과_연수  256344 non-null  float64
 3   배란_자극_여부               256344 non-null  int64  
 4   단일_배아_이식_여부            256344 non-null  float64
 5   착상_전_유전_검사_사용_여부       256344 non-null  float64
 6   착상_전_유전_진단_사용_여부       256344 non-null  float64
 7   남성_주_불임_원인             256344 non-null  int64  
 8   남성_부_불임_원인             256344 non-null  int64  
 9   여성_주_불임_원인             256344 non-null  int64  
 10  여성_부_불임_원인             256344 non-null  int64  
 11  부부_주_불임_원인             256344 non-null  int64  
 12  부부_부_불임_원인             256344 non-null  int64  
 13  불명확_불임_원인              256344 non-null  int64  
 14  불임_원인_-_난관_질환          256344 non-nu

In [1743]:
import pandas as pd
import numpy as np

# 파생 변수 생성 함수
def create_features(df):
    # 총 배아 생성 비율
    df['생성_배아_대비_이식_배아_비율'] = np.where(df['총_생성_배아_수'] != 0, df['이식된_배아_수'] / df['총_생성_배아_수'], -1)

    # 미세주입 배아 이식 비율
    df['ICSI_배아_이식_비율'] = np.where(df['미세주입에서_생성된_배아_수'] != 0, df['미세주입_배아_이식_수'] / df['미세주입에서_생성된_배아_수'], -1)

    # 이식된 배아 중 ICSI 배아가 이식된 비율
    df['이식된_배아중_ICSI_배아_비율'] = np.where(df['이식된_배아_수'] != 0, df['미세주입에서_생성된_배아_수'] / df['이식된_배아_수'], -1)

    # 저장된 배아 비율
    df['저장된_배아_비율'] = np.where(df['총_생성_배아_수'] != 0, df['저장된_배아_수'] / df['총_생성_배아_수'], -1)
    
    # 소수점 둘째 자리까지 반올림
    df = df.round(2)
    
    # inf, -inf 값을 -1로 대체
    df.replace([np.inf, -np.inf], -1, inplace=True)
    
    return df

# 파생 변수 생성
train = create_features(train)
test = create_features(test)

### 68. 임신_성공_여부

해당 시술로부터 임신 성공 여부

In [1744]:
value_counts_ratio(train, '임신_성공_여부')


임신_성공_여부별 임신_성공_여부 비율 및 갯수

            0    1         0        1   Total
variable                                     
0         1.0  0.0  190118.0      0.0  190118
1         0.0  1.0       0.0  66226.0   66226


## 변수 상태 확인

In [1745]:
train.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256344 entries, 0 to 256343
Data columns (total 96 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   ID                     256344 non-null  object 
 1   시술_당시_나이               256344 non-null  object 
 2   임신_시도_또는_마지막_임신_경과_연수  256344 non-null  float64
 3   배란_자극_여부               256344 non-null  int64  
 4   단일_배아_이식_여부            256344 non-null  float64
 5   착상_전_유전_검사_사용_여부       256344 non-null  float64
 6   착상_전_유전_진단_사용_여부       256344 non-null  float64
 7   남성_주_불임_원인             256344 non-null  int64  
 8   남성_부_불임_원인             256344 non-null  int64  
 9   여성_주_불임_원인             256344 non-null  int64  
 10  여성_부_불임_원인             256344 non-null  int64  
 11  부부_주_불임_원인             256344 non-null  int64  
 12  부부_부_불임_원인             256344 non-null  int64  
 13  불명확_불임_원인              256344 non-null  int64  
 14  불임_원인_-_난관_질환          256344 non-nu

C:\Users\juneh\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:1: FutureWarning: null_counts is deprecated. Use show_counts instead
  """Entry point for launching an IPython kernel.


In [1746]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90067 entries, 0 to 90066
Data columns (total 95 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   ID                     90067 non-null  object 
 1   시술_당시_나이               90067 non-null  object 
 2   임신_시도_또는_마지막_임신_경과_연수  90067 non-null  float64
 3   배란_자극_여부               90067 non-null  int64  
 4   단일_배아_이식_여부            90067 non-null  float64
 5   착상_전_유전_검사_사용_여부       90067 non-null  float64
 6   착상_전_유전_진단_사용_여부       90067 non-null  float64
 7   남성_주_불임_원인             90067 non-null  int64  
 8   남성_부_불임_원인             90067 non-null  int64  
 9   여성_주_불임_원인             90067 non-null  int64  
 10  여성_부_불임_원인             90067 non-null  int64  
 11  부부_주_불임_원인             90067 non-null  int64  
 12  부부_부_불임_원인             90067 non-null  int64  
 13  불명확_불임_원인              90067 non-null  int64  
 14  불임_원인_-_난관_질환          90067 non-null  int64  
 15  불임

In [1573]:
# # 결측값을 -1 으로 채우기
# train.fillna(-1, inplace=True)
# test.fillna(-1, inplace=True)

## 데이터 저장

In [1747]:
# CSV 파일 저장
train.to_csv('../data/Total_train_dataset_43.csv', index=False, encoding='utf-8-sig')
test.to_csv('../data/Total_test_dataset_43.csv', index=False, encoding='utf-8-sig')

.